# Paper Title: Direct and Indirect Hydrogen Storage: Dynamics of Interactions Within Europe's Highly Renewable Energy System

# Figure Plot Code

# Author: Zhiyuan Xie

# URL: 

In [ ]:
# Introduce necessary package

import pypsa
import pandas as pd
import numpy as np
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import matplotlib.lines as mlines
import matplotlib.patches as mpatches
import seaborn as sns
import csv
import pywt
import fatpack

from matplotlib.colors import ListedColormap
from matplotlib.patches import Rectangle
from matplotlib.patches import Patch
from matplotlib.patches import Polygon
from mpl_toolkits.mplot3d import Axes3D

from scipy import stats
from scipy.stats import norm, expon, weibull_min, kstest
from scipy.signal import cwt, morlet2
from scipy.spatial import ConvexHull
from scipy.stats import poisson
from scipy.stats import norm
from brokenaxes import brokenaxes


In [ ]:
# Import data

n_2020 = pypsa.Network('/Users/...')
n_2025 = ...
n_2030 = ...
n_2035 = ...
n_2040 = ...
n_2045 = ...
n_2050 = ...


In [ ]:
###################### Define Function #############################
####################################################################

In [ ]:
# Function 1: Calculate FFT

def spectrum1(h, dt=1):
    nt = len(h)
    npositive = nt//2
    pslice = slice(1, npositive)
    freqs = np.array([1/f for f in np.arange(0,dt,1/(nt))])[pslice]
    ft = np.fft.fft(h)[pslice]
    psraw = (np.abs(ft)/nt) ** 2

    # Convert PS to Power Spectral Density
    psdraw = psraw * dt * nt  # nt * dt is record length
    return freqs, psraw, psdraw


In [ ]:
# Function 2: read data & calculate the mean, std and high for each data & clean data

def basiccalculate_DNK(data):
    EleP = pd.DataFrame(data.buses_t.marginal_price["DK"].reset_index(drop=True))
    mean = EleP.mean().values[0]
    std = EleP.std().values[0]
    high = mean + 3*std
    EP = EleP[EleP.iloc[:,0] < high].reset_index(drop=True)
    return EleP, mean, std, high, EP

def basiccalculate_DEU(data):
    EleP = pd.DataFrame(data.buses_t.marginal_price["DE"].reset_index(drop=True))
    mean = EleP.mean().values[0]
    std = EleP.std().values[0]
    high = mean + 3*std
    EP = EleP[EleP.iloc[:,0] < high].reset_index(drop=True)
    return EleP, mean, std, high, EP

def basiccalculate_ITA(data):
    EleP = pd.DataFrame(data.buses_t.marginal_price["IT"].reset_index(drop=True))
    mean = EleP.mean().values[0]
    std = EleP.std().values[0]
    high = mean + 3*std
    EP = EleP[EleP.iloc[:,0] < high].reset_index(drop=True)
    return EleP, mean, std, high, EP

def basiccalculate_Real(data):
    EleP = pd.DataFrame(data.reset_index(drop=True))
    mean = EleP.mean().values[0]
    std = EleP.std().values[0]
    high = mean + 3*std
    EP = EleP[EleP.iloc[:,0] < high].reset_index(drop=True)
    return EleP, mean, std, high, EP


In [ ]:
# Function 3: opening time index for the storage charge and discharge

def P0time(data,chargethreshold):
    time_charge = data
    time_charge[time_charge.iloc[:,0] < chargethreshold] = 0
    time_charge[time_charge.iloc[:,0] >= chargethreshold] = 1
    return time_charge

def P1time(data,chargethreshold):
    time_charge = data
    time_charge[time_charge.iloc[:,0] > chargethreshold] = 0
    time_charge[time_charge.iloc[:,0] <= chargethreshold] = 1
    return time_charge


In [ ]:
# Function 4: Output the power flow of H2, actually now it can calculate for all the storage

def H2Link(data, charge_efficiency, discharge_efficiency):
    data_charge = []
    data_discharge = []
    
    # Calculate the first value differently if needed
    if (data.iloc[0] - data.iloc[-1]) > 0:
        data_charge.append((data.iloc[0] - data.iloc[-1]) / charge_efficiency)
        data_discharge.append(0)
    else:
        data_charge.append(0)
        data_discharge.append((data.iloc[-1] - data.iloc[0]) * discharge_efficiency)
    
    # Loop through the data series
    for i in range(1, len(data)):
        if (data.iloc[i] - data.iloc[i-1]) > 0:
            data_charge.append((data.iloc[i] - data.iloc[i-1]) / charge_efficiency)
            data_discharge.append(0)
        elif (data.iloc[i] - data.iloc[i-1]) < 0:
            data_charge.append(0)
            data_discharge.append((data.iloc[i-1] - data.iloc[i]) * discharge_efficiency)
        else:
            data_charge.append(0)
            data_discharge.append(0)
    
    # Convert the lists to Pandas Series
    data_charge_series = pd.Series(data_charge)
    data_discharge_series = pd.Series(data_discharge)
    
    return data_charge_series, data_discharge_series


In [ ]:
# Function 5: Calculate the ration of the data

def RationCal(data):
    deno_number = 3
    Ration_Cal = np.array([])
    for i in range(len(data)):
        if i < deno_number and i > 0:
            Ration_Cal = np.append(Ration_Cal,(data[i:(i+1)].mean()/((data[0:i].mean()+data[(i+1):(i+deno_number+1)].mean())/2)))
        elif i == 0:
            Ration_Cal = np.append(Ration_Cal,(data[0:1].mean()/(data[1:(deno_number+1)].mean())))
        elif i == (len(data)-1):
            Ration_Cal = np.append(Ration_Cal,(data[(len(data)-1):len(data)].mean()/(data[(len(data)-deno_number-1):(len(data)-1)].mean())))
        elif i < (len(data)-1) and i > (len(data)-4):
            Ration_Cal = np.append(Ration_Cal,(data[i:(i+1)].mean()/((data[(i-deno_number):i].mean()+data[(i+1):len(data)].mean())/2)))
        else:
            Ration_Cal = np.append(Ration_Cal,(data[i:(i+1)].mean()/((data[(i-deno_number):i].mean()+data[i+1:(i+deno_number+1)].mean())/2)))
    return Ration_Cal


In [ ]:
# Function 6: filling level

# suit for Denmark and Germany
# data_01 stands for H2 Store tank, data_02 stands for H2 Store underground, data_03 stands for battery.
def filling_01(data_01,data_02,data_03,data_01OPT,data_02OPT,data_03OPT):
    H2_Energy = data_01 + data_02
    Battery_Energy = data_03
    H2_OPT = data_01OPT + data_02OPT
    Battery_OPT = data_03OPT
    H2_Filling = H2_Energy/H2_OPT
    Battery_Filling = Battery_Energy/Battery_OPT
    return H2_Filling,Battery_Filling


# suit for Italy
# data_01 stands for H2 Store tank, data_02 stands for battery.
def filling_02(data_01,data_02,data_01OPT,data_02OPT):
    H2_Energy = data_01
    Battery_Energy = data_02
    H2_OPT = data_01OPT
    Battery_OPT = data_02OPT
    H2_Filling = H2_Energy/H2_OPT
    Battery_Filling = Battery_Energy/Battery_OPT
    return H2_Filling,Battery_Filling


In [ ]:
# Function 7: mark the waiting time of storage, V2

def process_series(data_series):
    new_data = [0]
    i = 1
    while i < len(data_series) - 1:
        prev_value = data_series[i - 1]
        current_value = data_series[i]
        next_value = data_series[i + 1]

        # situation 1,2,3,4
        if abs(current_value - next_value) < 0.0002:
            start_idx = i

            while i < len(data_series) - 1 and abs(data_series[i] - data_series[i + 1]) < 0.0002:
                i += 1

            end_idx = i

            if prev_value < current_value and end_idx + 1 < len(data_series) and (data_series[end_idx + 1] < current_value):
                new_data.extend([3] + [3] * (end_idx - start_idx))
            elif prev_value > current_value and end_idx + 1 < len(data_series) and (data_series[end_idx + 1] > current_value):
                new_data.extend([-3] + [-3] * (end_idx - start_idx))
            elif prev_value < current_value:
                new_data.extend([2] + [2] * (end_idx - start_idx))
            elif prev_value > current_value:
                new_data.extend([-2] + [-2] * (end_idx - start_idx))

            i += 1

        else:
            prev_value_01 = data_series[i - 1]
            current_value_01 = data_series[i]
            next_value_01 = data_series[i + 1]

            # situation 5, charging busy
            if (current_value_01 - next_value_01) < -0.0002:
                start01_idx = i

                while i < len(data_series) - 1 and (data_series[i] - data_series[i + 1]) < -0.0002:
                    i += 1

                end01_idx = i
                new_data.extend([1] + [1] * (end01_idx - start01_idx))
                i += 1

            # situation 6, discharge busy    
            elif (current_value_01 - next_value_01) > 0.0002:
                start02_idx = i

                while i < len(data_series) - 1 and (data_series[i] - data_series[i + 1]) > 0.0002:
                    i += 1

                end02_idx = i
                new_data.extend([-1] + [-1] * (end02_idx - start02_idx))
                i += 1

            else:
                new_data.append(0)
                i += 1

    #new_data.append(0)
    
    return pd.Series(new_data)



In [ ]:
# Function 8: Calculate the key parameter of EP

def calculate_statistics(data):
    means = [np.mean(data[i].values) for i in range(7)]
    std_ranges = [np.std(data[i].values) for i in range(7)]
    p97_5_minus_p2_5 = [np.percentile(data[i].values, 97.5) - np.percentile(data[i].values, 2.5) for i in range(7)]
    cv = [np.std(data[i].values)*100 / np.mean(data[i].values) for i in range(7)]
    return means, std_ranges, p97_5_minus_p2_5, cv

def calculate_statistics_real(data):
    means = [np.mean(data[i].values) for i in range(8)]
    std_ranges = [np.std(data[i].values) for i in range(8)]
    p97_5_minus_p2_5 = [np.percentile(data[i].values, 97.5) - np.percentile(data[i].values, 2.5) for i in range(8)]
    cv = [np.std(data[i].values)*100 / np.mean(data[i].values) for i in range(8)]
    return means, std_ranges, p97_5_minus_p2_5, cv


In [ ]:
# Function 9: Calculate the price

def create_price_variables(year, charge_h2, direct_discharge_h2, indirect_discharge_h2, charge_battery, discharge_battery, elep, hp):
    
    price_charge_h2 = np.multiply(np.array(charge_h2), np.array(elep.iloc[:,0]))
    price_direct_discharge_h2 = np.multiply(np.array(direct_discharge_h2), np.array(hp))
    price_indirect_discharge_h2 = np.multiply(np.array(indirect_discharge_h2), np.array(hp))
    price_charge_battery = np.multiply(np.array(charge_battery), np.array(elep.iloc[:,0]))
    price_discharge_battery = np.multiply(np.array(discharge_battery), np.array(elep.iloc[:,0]))
    
    return price_charge_h2, price_direct_discharge_h2, price_indirect_discharge_h2, price_charge_battery, price_discharge_battery


In [ ]:
# Function 10: Calculate the cost

def create_cost_variables(year, price_charge_battery, price_discharge_battery, link_charge_battery, link_discharge_battery):

    cost_charge_battery = np.multiply(price_charge_battery,np.array(link_charge_battery.iloc[:,0]))
    cost_discharge_battery = np.multiply(price_discharge_battery,np.array(link_discharge_battery.iloc[:,0]))
    return cost_charge_battery, cost_discharge_battery   


In [ ]:
# Function 11: Cycle Capture Method

def calculate_all_cycles(series,threshold_01,threshold_02):
    df = pd.DataFrame(series)
    df.columns = ['filling_level']

    df['difference'] = df['filling_level'].diff()

    df['state'] = 'waiting'
    df.loc[df['difference'] > 0.0002, 'state'] = 'charging'
    df.loc[df['difference'] < -0.0002, 'state'] = 'discharging'

    cycles = []
    
    # New Series to track state within each cycle
    state_series = pd.Series('waiting', index=series.index)
    
    current_cycle = {'start': None, 'end': None, 'length': 0, 'net_energy_charging': 0, 'net_energy_discharging': 0, 'min_charging': 0, 'max_charging': 0, 'min_discharging': 0, 'max_discharging': 0}
    charging = False
    discharging = False
    first_state = ''

    for index, row in df.iterrows():
        state = row['state']
        
        state_series.loc[index] = state
        
        filling_level = row['filling_level']
        diff = row['difference']

        # set the first_state that can help us entry the judge condition
        if not first_state and state != 'waiting':
            first_state = state
            current_cycle['start'] = series.index.get_loc(index)     
            
        # situation 1, begin with charging
        if first_state == 'charging':
            if state == 'charging':
                
                if not charging and not discharging:
                    current_cycle['start'] = series.index.get_loc(index)
                    charging = True
                elif discharging:
                    charging = True
                    discharging = False
                if current_cycle['min_charging'] == 0:
                    current_cycle['min_charging'] = filling_level
                    current_cycle['max_charging'] = filling_level
                else:
                    current_cycle['min_charging'] = min(current_cycle['min_charging'], filling_level)
                    current_cycle['max_charging'] = max(current_cycle['max_charging'], filling_level)
                
                if (current_cycle['max_charging'] - current_cycle['min_charging'] > threshold_01) and (current_cycle['max_discharging'] - current_cycle['min_discharging'] > threshold_02):
                    current_cycle['end'] = series.index.get_loc(index)
                    current_cycle['length'] = series.index.get_loc(index) - current_cycle['start']
                    current_cycle['net_energy'] = current_cycle['net_energy_charging'] - current_cycle['net_energy_discharging']
                    cycles.append(current_cycle)
                    current_cycle = {'start': series.index.get_loc(index), 'end': None, 'length': 0, 'net_energy': 0, 'net_energy_charging': 0, 'net_energy_discharging': 0, 'min_charging': 0, 'max_charging': 0, 'min_discharging': 0, 'max_discharging': 0}
                    charging = False
                    discharging = False
                
                current_cycle['net_energy_charging'] += abs(diff) if diff > 0 else 0
            
            elif state == 'discharging' and charging:
                discharging = True
                current_cycle['net_energy_discharging'] += abs(diff) if diff < 0 else 0
                if current_cycle['min_discharging'] == 0:
                    current_cycle['min_discharging'] = filling_level
                    current_cycle['max_discharging'] = filling_level
                else:
                    current_cycle['min_discharging'] = min(current_cycle['min_discharging'], filling_level)
                    current_cycle['max_discharging'] = max(current_cycle['max_discharging'], filling_level)

        # situation 2, begin with discharging
        elif first_state == 'discharging':
            if state == 'discharging':
                
                if not charging and not discharging:
                    current_cycle['start'] = series.index.get_loc(index)
                    discharging = True
                elif charging:
                    discharging = True
                    charging = False
                if current_cycle['min_discharging'] == 0:
                    current_cycle['min_discharging'] = filling_level
                    current_cycle['max_discharging'] = filling_level
                else:
                    current_cycle['min_discharging'] = min(current_cycle['min_discharging'], filling_level)
                    current_cycle['max_discharging'] = max(current_cycle['max_discharging'], filling_level)
                
                if (current_cycle['max_charging'] - current_cycle['min_charging'] > threshold_01) and (current_cycle['max_discharging'] - current_cycle['min_discharging'] > threshold_02):
                    current_cycle['end'] = series.index.get_loc(index)
                    current_cycle['length'] = series.index.get_loc(index) - current_cycle['start']
                    current_cycle['net_energy'] = current_cycle['net_energy_charging'] - current_cycle['net_energy_discharging']
                    cycles.append(current_cycle)
                    current_cycle = {'start': series.index.get_loc(index), 'end': None, 'length': 0, 'net_energy': 0, 'net_energy_charging': 0, 'net_energy_discharging': 0, 'min_charging': 0, 'max_charging': 0, 'min_discharging': 0, 'max_discharging': 0}
                    charging = False
                    discharging = False
                
                current_cycle['net_energy_discharging'] += abs(diff) if diff < 0 else 0
            
            elif state == 'charging' and discharging:
                discharging = True
                current_cycle['net_energy_charging'] += abs(diff) if diff > 0 else 0
                if current_cycle['min_charging'] == 0:
                    current_cycle['min_charging'] = filling_level
                    current_cycle['max_charging'] = filling_level
                else:
                    current_cycle['min_charging'] = min(current_cycle['min_charging'], filling_level)
                    current_cycle['max_charging'] = max(current_cycle['max_charging'], filling_level)

    new_series = pd.Series(0, index=series.index)

    for cycle in cycles:
        start_index = cycle['start']
        end_index = cycle['end']
        length = cycle['length']
        new_series.iloc[start_index:end_index] = length

    return cycles, new_series, state_series



In [ ]:
######################## Basic Calculation #############################
########################################################################

In [ ]:
## Calculate the electricity price from PyPSA

# DNK
DNK_EleP_2020, DNK_mean_2020, DNK_std_2020, DNK_High_2020, DNK_EP_2020 = basiccalculate_DNK(n_2020)

# DEU
...

# ITA
...

In [ ]:
## Read & Calculate the real electricity price

# read data from csv and divide it, DNK
Real_DNKEP_all = pd.read_csv("....csv")

#DEU
...

#ITA
...

In [ ]:
# Prepare the H2 marginal price data

# DNK H2 Marginal Price

DNK_H2_MarginalPrice_2020 = n_2020.buses_t.marginal_price["DK H2"]
DNK_H2_MarginalPrice_2025 = ...

# DEU H2 Marginal Price
...

# ITA H2 Marginal Price
...


In [ ]:
#################### Capture the character of EP ######################
#######################################################################

In [ ]:
# Get EP percentile

#DNK
DNK_Duration_10 = [DNK_EP_2020.quantile(0.1).values[0],...]
DNK_Duration_20 = ...

#DEU
...

#ITA
...

# Get H2 Marginal Price percentile

#DNK
DNK_H2_Duration_10 = [DNK_H2_MarginalPrice_2020.quantile(0.1),...]
...

#DEU
...

#ITA
...

In [ ]:
# Plot Figure 05 and S03

year = [2020,2025,2030,2035,2040,2045,2050]
# Plot setup
font_axis = {'family':'Times New Roman','weight':'normal','size':16}
font_title = {'family' : 'Times New Roman','weight' : 'normal', 'size' : 13}

# Create a 3x2 subplot grid
fig, axs = plt.subplots(3, 2, figsize=(15,15))  # Adjusted figsize for 3x2 grid

axs[0, 0].plot(year, DNK_Duration_10, color = "#F9E2AF", linewidth=2.0, marker=".", markersize=10)
axs[0, 0]....
axs[0, 0].set_title('Percentile of DNK Electricity Price', font_title)
#axs[0, 0].set_xlabel('Year', font_axis)
axs[0, 0].set_ylabel('Electricity Price (€/MWh)', font_axis)
#axs[0, 0].set_xticks(year, prop = {'family':'Times New Roman','size':13})
axs[0, 0].set_xticklabels([])
axs[0, 0].set_yticks(np.linspace(0,400,11), prop = {'family':'Times New Roman','size':13})
axs[0, 0].legend(frameon=False, prop = {'family':'Times New Roman','size':13})

axs[1, 0].plot(year, DEU_Duration_10, color = "#F9E2AF", linewidth=2.0, marker=".", markersize=10)
axs[1, 0]....
axs[1, 0].set_title('Percentile of DEU Electricity Price', font_title)
#axs[1, 0].set_xlabel('Year', font_axis)
axs[1, 0].set_ylabel('Electricity Price (€/MWh)', font_axis)
#axs[1, 0].set_xticks(year, prop = {'family':'Times New Roman','size':13})
axs[1, 0].set_xticklabels([])
axs[1, 0].set_yticks(np.linspace(0,400,11), prop = {'family':'Times New Roman','size':13})
#axs[1, 0].legend(frameon=False, prop = {'family':'Times New Roman','size':13})

axs[2, 0].plot(year, ITA_Duration_10, color = "#F9E2AF", linewidth=2.0, marker=".", markersize=10)
axs[2, 0]....
axs[2, 0].set_title('Percentile of ITA Electricity Price', font_title)
axs[2, 0].set_xlabel('Year', font_axis)
axs[2, 0].set_ylabel('Electricity Price (€/MWh)', font_axis)
axs[2, 0].set_xticks(year, prop = {'family':'Times New Roman','size':13})
axs[2, 0].set_yticks(np.linspace(0,400,11), prop = {'family':'Times New Roman','size':13})
#axs[2, 0].legend(frameon=False, prop = {'family':'Times New Roman','size':13})

axs[0, 1].plot(year, DNK_H2_Duration_10, label="10%", color = "#F9E2AF", linewidth=2.0, marker=".", markersize=10)
axs[0, 1]....
axs[0, 1].set_title('Percentile of DNK H2 Marginal Price', font_title)
#axs[0, 1].set_xlabel('Year', font_axis)
axs[0, 1].set_ylabel('Hydrogen Price (€/MWh)', font_axis)
#axs[0, 1].set_xticks(year, prop = {'family':'Times New Roman','size':13})
axs[0, 1].set_xticklabels([])
axs[0, 1].set_yticks(np.linspace(0,400,11), prop = {'family':'Times New Roman','size':13})
axs[0, 1].legend(frameon=False, prop = {'family':'Times New Roman','size':13}, loc="upper left")

axs[1, 1].plot(year, DEU_H2_Duration_10, color = "#F9E2AF", linewidth=2.0, marker=".", markersize=10)
axs[1, 1]....
axs[1, 1].plot(year, DEU_H2_Duration_medium, color = "#393646", linewidth=4.0, linestyle='-.', marker=".", markersize=10)
axs[1, 1].set_title('Percentile of DEU H2 Marginal Price', font_title)
#axs[1, 1].set_xlabel('Year', font_axis)
axs[1, 1].set_ylabel('Hydrogen Price (€/MWh)', font_axis)
#axs[1, 1].set_xticks(year, prop = {'family':'Times New Roman','size':13})
axs[1, 1].set_xticklabels([])
axs[1, 1].set_yticks(np.linspace(0,400,11), prop = {'family':'Times New Roman','size':13})
#axs[1, 1].legend(frameon=False, prop = {'family':'Times New Roman','size':13})

axs[2, 1].plot(year, ITA_H2_Duration_10, color = "#F9E2AF", linewidth=2.0, marker=".", markersize=10)
axs[2, 1]....
axs[2, 1].plot(year, ITA_H2_Duration_medium, color = "#393646", linewidth=4.0, linestyle='-.', marker=".", markersize=10)
axs[2, 1].set_title('Percentile of ITA Hydrogen Marginal Price', font_title)
axs[2, 1].set_xlabel('Year', font_axis)
axs[2, 1].set_ylabel('Hydrogen Price (€/MWh)', font_axis)
axs[2, 1].set_xticks(year, prop = {'family':'Times New Roman','size':13})
axs[2, 1].set_yticks(np.linspace(0,400,11), prop = {'family':'Times New Roman','size':13})
#axs[2, 1].legend(frameon=False, prop = {'family':'Times New Roman','size':13})


fig.subplots_adjust(hspace=0.15)

plt.savefig('Figure S03.png', dpi=300)

plt.show()


In [ ]:
# Plot Figure S04 and S05

# prepare the data
DNK_Eleprice_data = [Real_DNKEP_2019["DK"],...]
DEU_Eleprice_data = ...
ITA_Eleprice_data = ...

years = ['2019', '2020', '2025', '2030', '2035', '2040', '2045', '2050']

# Create subplots
fig, axs = plt.subplots(8, 3, figsize=(15, 20))  # 8 rows, 3 columns

datasets = {
    'DNK': DNK_Eleprice_data,
    'DEU': DEU_Eleprice_data,
    'ITA': ITA_Eleprice_data
}

# Calculate x-axis and y-axis limits for each country
x_limits = {}
y_limits = {}

for country, data in datasets.items():
    x_limits[country] = max([np.percentile(dataset, 95) for dataset in data])
    if x_limits[country] < 250:
        x_limits[country] = 200
    
    # Get histogram values for the eighth year's data without plotting
    counts, bin_edges = np.histogram(data[-1], bins=30)
    y_limits[country] = sorted(counts)[-1]  # Second largest count
    if y_limits[country] < 3000:
        y_limits[country] = 3000

# Plot data with uniform x and y axes for each country
for row, year in enumerate(datasets['DNK']):
    for col, (country, data) in enumerate(datasets.items()):
        axs[row, col].hist(data[row], bins=np.linspace(0, x_limits[country], 31), edgecolor='black', alpha=0.7)
        axs[row, col].set_title(f'{country} - Electricity Price in {years[row]}')
        axs[row, col].set_xlabel('Electricity Price')
        axs[row, col].set_ylabel('Frequency')
        axs[row, col].set_xlim(0, x_limits[country])
        axs[row, col].set_ylim(0, y_limits[country])

plt.tight_layout()

plt.savefig('Figure S04.png', dpi = 300)

plt.show()


In [ ]:
#################### Plot the map of the model results ######################
#############################################################################

In [ ]:
######## prepare CO2 emission data

# Calculate each country CO2 emission

# Countries and their codes
countries = {"DNK": "DK", "DEU": "DE", "ITA": "IT"}

# Emission factors
emission_factors = {
    "coal": 0.336,
    "lignite": 0.407,
    "gas": 0.201,
    "oil": 0.266,
}

# Years to consider
years = [2020,2025,2030,2035,2040,2045,2050]

CO2_emissions = {}

for year in years:
    n_year = eval("n_{}".format(year))

    bus0_df = n_year.links["bus0"].to_frame()
    #bus0_df['name'] = bus0_df.index

    for country_code, country in countries.items():

        for fuel, emission_factor in emission_factors.items():
            
            keyword = f"{country} {fuel}"

            matched_rows = bus0_df[bus0_df["bus0"].str.contains(keyword)]

            matched_names = matched_rows.index.tolist()

            for name in matched_names:
                fuel_consumption = n_year.links_t["p0"][name].sum()
                CO2_emissions[(country_code, fuel, year)] = fuel_consumption * emission_factor

total_CO2_emissions = {}

for country_code in countries:
    for year in years:
        total_CO2_emissions[(country_code, year)] = sum(value for key, value in CO2_emissions.items() if key[0] == country_code and key[2] == year)


# Distribution CO2 emission value

# DNK, CO2
DNK_CO2emission_2020 = total_CO2_emissions[("DNK",2020)]
DNK_CO2emission_2025 = ...

# DEU, CO2
...

# ITA, CO2
...


In [ ]:
######## prepare capacity of Fuel Cell, Electrolysis, Sabatier, Battery, H2 Storage

#### Capacity of Battery & H2 Storage
# OPT Storage capacity for DNK
DNK_H2_OPT_2020 = n_2020.stores.e_nom_opt["DK H2 Store tank"] + n_2020.stores.e_nom_opt["DK H2 Store underground"]
DNK_H2_OPT_2025 = ...

# OPT Storage capacity for DEU
...

# OPT Storage capacity for ITA
...

#### Capacity of Fuel Cell

# DNK
DNK_H2_FuelCell_pnom_2020 = n_2020.links.p_nom_opt["DK H2 Fuel Cell"]
DNK_H2_FuelCell_pnom_2025 = ...

# DEU
...

# ITA
...

#### Capacity of Electrolysis

# DNK
DNK_H2_Electrolysis_pnom_2020 = n_2020.links.p_nom_opt["DK H2 Electrolysis"]
DNK_H2_Electrolysis_pnom_2025 = ...

# DEU
...

# ITA
...

#### Capacity of Sabatier

# DNK
DNK_H2_Sabatier_pnom_2020 = n_2020.links.p_nom_opt["DK Sabatier"]
DNK_H2_Sabatier_pnom_2025 = ...

# DEU
...

# ITA
...


In [ ]:
######## prepare Wind & Solar generation & Curtailment

# Wind and Solar Capacity

# DNK
DNK_onwind_OPT = [n_2020.generators.loc["DK onwind"]["p_nom_opt"],...]
DNK_offwind_OPT = [n_2020.generators.loc["DK offwind"]["p_nom_opt"],...]
DNK_solar_OPT = [n_2020.generators.loc["DK solar"]["p_nom_opt"],...]

# DEU
...

# ITA
...

# Wind and Solar Curtailment

# DNK
## wind
DNK_windgenerators_2020 = n_2020.generators_t["p"]["DK onwind"].sum() + n_2020.generators_t["p"]["DK offwind"].sum()
DNK_windgenerators_2025 = ...

DNK_WindMax_2020 = (n_2020.generators_t.p_max_pu["DK onwind"].multiply(n_2020.generators.p_nom_opt["DK onwind"])).sum() + (n_2020.generators_t.p_max_pu["DK offwind"].multiply(n_2020.generators.p_nom_opt["DK offwind"])).sum()
DNK_WindMax_2025 = ...

DNK_Wind_Curtail_2020 = (DNK_WindMax_2020-DNK_windgenerators_2020)*100/DNK_WindMax_2020
DNK_Wind_Curtail_2025 = ...
                                                                                       
## solar
DNK_solargenerators_2020 = n_2020.generators_t["p"]["DK solar"].sum()
DNK_solargenerators_2025 = ...

DNK_SolarMax_2020 = (n_2020.generators_t.p_max_pu["DK solar"].multiply(n_2020.generators.p_nom_opt["DK solar"])).sum()                                                                                                                          
DNK_SolarMax_2025 = ...                                                                      

DNK_Solar_Curtail_2020 = (DNK_SolarMax_2020-DNK_solargenerators_2020)*100/DNK_SolarMax_2020
DNK_Solar_Curtail_2025 = ...
                                                                                                                                  
# DEU
## wind
...

## solar
...

# ITA
## wind
...

## solar
...


In [ ]:
######## prepare the accumulative transmission value

# Transmission Value

# DNK, transimission
DNK_transmission_2020 = [-n_2020.links_t["p0"]["DK-NL"].sum(), ...]
DNK_transmission_2025 = ...

# DEU, transimission
...

# ITA, transmission
...

# Transmission accumulated

DNK_DKNL = [DNK_transmission_2020[0], ...]
...

# Seperate the positive and negative value

def calculate_positive_and_negative_totals(values):
    pos_total = np.array([val if val > 0 else 0 for val in values])
    neg_total = np.array([val if val < 0 else 0 for val in values])
    return pos_total, neg_total


DNK_DKNL_pos, DNK_DKNL_neg = calculate_positive_and_negative_totals(DNK_DKNL)
...

# cumulative

# DNK
cumulative_DNK_DKNL_pos = [0,0,0,0,0,0,0]
cumulative_DNK_SEDK_pos = cumulative_DNK_DKNL_pos + DNK_DKNL_pos
cumulative_DNK_NODK_pos = ...

# DEU
...

#ITA
...


In [ ]:
######## Plot Figure 04 and S2

# Basic sets
font_axis = {'family':'Times New Roman','weight':'normal','size':16}
font_title = {'family' : 'Times New Roman','weight' : 'normal', 'size' : 13}
year = [2020,2025,2030,2035,2040,2045,2050]
locations = ['DNK', 'DEU', 'ITA']

# Adjusted figure size
fig, axs = plt.subplots(3, 3, figsize=(25,20))

# Combine the data

capacity_storage_data = [(DNK_H2_FuelCell_pnom, ...),
                         (DEU_H2_FuelCell_pnom, ...),
                         (ITA_H2_FuelCell_pnom, ...)]

CO2emission_data = [DNK_CO2emission, ...]

DNK_EleLoad_Average = n_2020.loads_t.p["DK"].sum()
DEU_EleLoad_Average = n_2020.loads_t.p["DE"].sum()
ITA_EleLoad_Average = n_2020.loads_t.p["IT"].sum()
averages = [DNK_EleLoad_Average, DEU_EleLoad_Average, ITA_EleLoad_Average]

generation_data = [(DNK_windgenerators, DNK_solargenerators), (DEU_windgenerators, DEU_solargenerators), (ITA_windgenerators, ITA_solargenerators)]
wind_curtailment = [DNK_Wind_Curtail, DEU_Wind_Curtail, ITA_Wind_Curtail]
solar_curtailment = [DNK_Solar_Curtail, DEU_Solar_Curtail, ITA_Solar_Curtail]

normalized_generation_data = [([x/averages[i] for x in pair[0]], [y/averages[i] for y in pair[1]]) for i, pair in enumerate(generation_data)]

labels = {
    "DK-NL": "NL", "SE-DK": "SE", "NO-DK": "NO", "DE-DK": "DE",
    ...
}

# Plot

for i, (fuelcell, electrolysis, sabatier, h2, battery) in enumerate(capacity_storage_data):
    
    axs[i, 0].plot(year, h2, label=f"{locations[i]} H$_{2}$ Tank & Underground Storage Capacity", color="#FF7000", linewidth=5.0, marker=".", markersize=25)
    axs[i, 0].plot(year, battery, label=f"{locations[i]} Battery Capacity", color="#025464", linewidth=5.0, marker=".", markersize=25)
    axs[i, 0].set_title('H$_{2}$ & Battery Revelant Technology Capacity and CO$_{2}$ emission', font_title)
    axs[i, 0].set_xlabel('Year', font_axis)
    axs[i, 0].set_ylabel('Storage Capacity (MWh)', font_axis)
    axs[i, 0].set_xticks(year)
    axs[i, 0].set_ylim([0, 710000])
    axs[i, 0].legend(frameon=False, prop={'family':'Times New Roman', 'size':13}, loc="upper left")
    
    # Add right y-axis for renewable penetration
    ax2 = axs[i, 0].twinx()
    ax2.plot(year, fuelcell, label=f"{locations[i]} FuelCell Capacity", color="#FFBF00", linewidth=5.0, marker=".", markersize=25)
    ax2.plot(year, electrolysis, label=f"{locations[i]} Electrolysis Capacity", color="#10A19D", linewidth=5.0, marker=".", markersize=25)
    ax2.plot(year, sabatier, label=f"{locations[i]} Sabatier Capacity", color="#540375", linewidth=5.0, marker=".", markersize=25)
    
    #ax2.yaxis.set_label_position('right')
    #ax2.yaxis.set_ticks_position('right')
    ax2.set_ylabel('Electrolysis, Sabatier & FuelCell Capacity (MW)', font_axis)
    ax2.set_ylim([0, 30000])
    
    # Create a third y-axis
    ax3 = ax2.twinx()

    # Move the third y-axis spine
    ax3.spines["right"].set_position(("axes", 1.18))  # Adjust as necessary
    ax3.set_frame_on(True)
    ax3.patch.set_visible(False)
    ax3.yaxis.set_label_position('right')
    ax3.yaxis.set_ticks_position('right')

    # Plot on the third y-axis
    ax3.plot(year, CO2emission_data[i], label=f"{locations[i]} CO$_{2}$ emission", color="#C1693C", linewidth=5.0, linestyle="dashed", marker=".", markersize=25)
    ax3.set_ylabel('CO$_{2}$ Emission (t)', font_axis)
    ax3.set_ylim([0, 190000000])  # Adjust as necessary
    
    ax2.yaxis.set_label_position('right')
    ax2.yaxis.set_ticks_position('right')

    
    # Create a common legend for all plots in the subfigure
    lines = axs[i, 0].get_lines() + ax2.get_lines() + ax3.get_lines()
    labels = [line.get_label() for line in lines]
    axs[i, 0].legend(lines, labels, frameon=False, prop={'family':'Times New Roman', 'size':13}, loc="upper left")

for i, (wind, solar) in enumerate(normalized_generation_data):
    axs[i, 1].plot(year, wind, label=f"{locations[i]} Wind Generation Load Normalization", color="#068DA9", linewidth=5.0, marker=".", markersize=25)
    axs[i, 1].plot(year, solar, label=f"{locations[i]} Solar Generation Load Normalization", color="#E55807", linewidth=5.0, marker=".", markersize=25)
    axs[i, 1].set_title('RE Generation & Curtailment', font_title)
    axs[i, 1].set_xlabel('Year', font_axis)
    axs[i, 1].set_ylabel('RE Generation Load Normalization (MWh/MWh_Load)', font_axis)
    axs[i, 1].set_xticks(year)
    axs[i, 1].set_ylim([0, 5])  # Set the same y limit

    # Add right y-axis for curtailment
    ax3 = axs[i, 1].twinx()
    ax3.plot(year, wind_curtailment[i], label=f"{locations[i]} Wind Curtail", color="#2E8B57", linewidth=5.0, linestyle="dashed", marker=".", markersize=25)
    ax3.plot(year, solar_curtailment[i], label=f"{locations[i]} Solar Curtail", color="#FFD700", linewidth=5.0, linestyle="dashed", marker=".", markersize=25)
    ax3.set_ylabel('Curtailment Percentile (%)', font_axis)
    ax3.set_ylim([0, 25])

    # Create a common legend for all plots in the subfigure
    lines = axs[i, 1].get_lines() + ax3.get_lines()
    labels = [line.get_label() for line in lines]
    axs[i, 1].legend(lines, labels, frameon=False, prop={'family':'Times New Roman', 'size':13}, loc="upper left")


bar_width = 2.5

axs[0, 2].barh(years, abs(DNK_DKNL_pos), left=cumulative_DNK_DKNL_pos, color="#F38181", height=bar_width, label='DNK_DKNL')
axs[0, 2]...

axs[0, 2].legend(frameon=False, prop={'family': 'Times New Roman', 'size': 13}, loc="upper left")
axs[0, 2].set_xlim([-40000000, 10000000])
axs[0, 2].set_title('DNK Import & Export Power Details', font_title)


axs[1, 2].barh(years, abs(DEU_BEDE_pos), left=cumulative_DEU_BEDE_pos, color="#F38181", height=bar_width, label='DEU_BEDE')
axs[1, 2]...

axs[1, 2].legend(frameon=False, prop={'family': 'Times New Roman', 'size': 13}, loc="upper left")
axs[1, 2].set_xlim([-90000000, 90000000])
axs[1, 2].set_title('DEU Import & Export Power Details', font_title)


axs[2, 2].barh(years, abs(ITA_GRIT_pos), left=cumulative_ITA_GRIT_pos, color="#F38181", height=bar_width, label='ITA_GRIT')
axs[2, 2]...

axs[2, 2].legend(frameon=False, prop={'family': 'Times New Roman', 'size': 13}, loc="upper left")
axs[2, 2].set_xlim([-80000000, 40000000])
axs[2, 2].set_title('ITA Import & Export Power Details', font_title)

axs[0, 2].axvline(0, color='black', linestyle='--')
axs[0, 2].set_xlabel('Energy transfered (MWh)', font_axis)
axs[0, 2].set_ylabel('Year', font_axis)

axs[1, 2].axvline(0, color='black', linestyle='--')
axs[1, 2].set_xlabel('Energy transfered (MWh)', font_axis)
axs[1, 2].set_ylabel('Year', font_axis)

axs[2, 2].axvline(0, color='black', linestyle='--')
axs[2, 2].set_xlabel('Energy transfered (MWh)', font_axis)
axs[2, 2].set_ylabel('Year', font_axis)


# Adjust spaces between subplots
fig.subplots_adjust(hspace=0.25, wspace=0.45)

plt.savefig('Figure S02.png',dpi=300)

plt.show()


In [ ]:
#################### Plot the original data time series ######################
##############################################################################

In [ ]:
# Including H2, Battery, Fuel Cell, Electrolysis, Sabatier Time Series Data

# DNK
DNK_H2_Change_2050 = n_2050.stores_t.e["DK H2 Store underground"] + n_2050.stores_t.e["DK H2 Store tank"]
DNK_Battery_Change_2050 = n_2050.stores_t.e["DK battery"]
DNK_FuelCell_Output_2050 = -n_2050.links_t.p1["DK H2 Fuel Cell"]
DNK_Electrolysis_Output_2050 = -n_2050.links_t.p1["DK H2 Electrolysis"]
DNK_Sabatier_Input_2050 = n_2050.links_t.p0["DK Sabatier"]

# DEU
...

# ITA
...

# Normalization

# def function
def normalize(series):
    return (series - series.min()) / (series.max() - series.min())

# DNK
DNK_H2_Change_2050_norm = normalize(DNK_H2_Change_2050)
...

# DEU
...

# ITA
...


In [ ]:
# Plot Figure 03 and S1

fig, axs = plt.subplots(5, 3, figsize=(11, 13))  # 5x3 subplots for 5 datasets across 3 countries

# Define a list of colors for the plots, labels, and datasets for each country
colors = ['#711DB0', '#3D30A2', '#C21292', '#EF4040', '#FFA732']
labels = ['H$_{2}$ Tank & \nUnderground Storage \nFilling Level', 'Battery Filling Level', 'FuelCell Output', 'Electrolysis Output', 'Sabatier Input']
countries = ['Denmark', 'Germany', 'Italy']
datasets = {
    'Denmark': [DNK_H2_Change_2050_norm, DNK_Battery_Change_2050_norm, DNK_FuelCell_Output_2050_norm, DNK_Electrolysis_Output_2050_norm, DNK_Sabatier_Input_2050_norm],
    'Germany': [DEU_H2_Change_2050_norm, DEU_Battery_Change_2050_norm, DEU_FuelCell_Output_2050_norm, DEU_Electrolysis_Output_2050_norm, DEU_Sabatier_Input_2050_norm],
    'Italy': [ITA_H2_Change_2050_norm, ITA_Battery_Change_2050_norm, ITA_FuelCell_Output_2050_norm, ITA_Electrolysis_Output_2050_norm, ITA_Sabatier_Input_2050_norm]
}

# Create custom legend entries
custom_legend_entries = [
    Line2D([], [], color='w', marker='o', markersize=15, markerfacecolor=colors[i], label=label)
    for i, label in enumerate(labels)
]

xticks = [0, 744, 1416, 2160, 2880, 3624, 4344, 5088, 5832, 6552, 7296, 8016, 8760]

for i, label in enumerate(labels):
    for j, country in enumerate(countries):
        series = normalize(datasets[country][i])
        ax = axs[i, j]
        ax.plot(series.values, color=colors[i], label=f"{country} {label}")

        if j == 0:
            ax.legend(handles=[custom_legend_entries[i]], loc='upper left', frameon=True, fontsize='11')
        
        if i == len(labels) - 1:
            ax.set_xticks(xticks)
            ax.set_xticklabels(['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec', 'End'], rotation=45, fontsize='11')
        else:
            ax.set_xticks([])
        
        if i < 4:
            ax.set_xticks([])
        else:
            ax.set_xticks(xticks)
            ax.set_xticklabels(['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec', 'End'], rotation=45, fontsize='11')
        ax.set_xlim(0, 8760)

plt.tight_layout()

plt.savefig('Figure S01.png', dpi=300)

plt.show()

In [ ]:
#################### Calculate the LCOS & Benefits of Storage ##########################
########################################################################################

In [ ]:
# Calculate the ratio between gas electricity and gas heating

def calculate_gas_transactions(network_data):
    # Helper function to check if any of the keywords is in the tech string
    def contains_keywords(tech, keywords):
        return any(keyword in tech for keyword in keywords)

    electricity_keywords = ["CCGT", "OCGT", "central gas CHP electric"]
    heat_keywords = ["central gas boiler", "decentral gas boiler"]

    gas_techs = network_data.links[network_data.links["bus0"].str.contains('gas')].index.tolist()

    gas_electricity_demand_techs = [tech for tech in gas_techs if contains_keywords(tech, electricity_keywords)]
    gas_heat_demand_techs = [tech for tech in gas_techs if contains_keywords(tech, heat_keywords)]

    total_gas_electricity_demand = sum(network_data.links_t.p0[tech].sum() for tech in gas_electricity_demand_techs if tech in network_data.links_t.p0)
    total_gas_heat_demand = sum(network_data.links_t.p0[tech].sum() for tech in gas_heat_demand_techs if tech in network_data.links_t.p0)

    filtered_rows_supply = network_data.links[network_data.links.index.str.contains('Sabatier')]
    gas_supplying_techs = filtered_rows_supply.index.tolist()
    total_gas_supplied = sum(-network_data.links_t.p1[tech].sum() for tech in gas_supplying_techs if tech in network_data.links_t.p1)
    total_gas_demand = total_gas_electricity_demand + total_gas_heat_demand

    return {
        'total_gas_electricity_demand': total_gas_electricity_demand,
        'total_gas_heat_demand': total_gas_heat_demand,
        'total_gas_supplied': total_gas_supplied,
        'total_gas_demand': total_gas_demand,
    }

gas_transactions = {}

networks = {
    2020: n_2020,
    2025: n_2025,
    2030: n_2030,
    2035: n_2035,
    2040: n_2040,
    2045: n_2045,
    2050: n_2050,
}

for year, network_data in networks.items():
    transactions = calculate_gas_transactions(network_data)
    gas_transactions[year] = transactions

for year, transactions in gas_transactions.items():

    if transactions['total_gas_demand'] > 0:
        proportion_gas_electricity = transactions['total_gas_electricity_demand'] / transactions['total_gas_demand']
    else:
        proportion_gas_electricity = 0

gas_electricity_ratios = {}

for year, data in gas_transactions.items():
    ratio = data['total_gas_electricity_demand'] / data['total_gas_demand']
    gas_electricity_ratios[year] = ratio

In [ ]:
#################### Get the flow of storage

In [ ]:
#DNK

years = [2020, 2025, 2030, 2035, 2040, 2045, 2050]

for year in years:
    n_year = globals()[f"n_{year}"]

    tanklink_charge, tanklink_discharge = H2Link(n_year.stores_t.e["DK H2 Store tank"],1,1)
    underlink_charge, underlink_discharge = H2Link(n_year.stores_t.e["DK H2 Store underground"],1,1)

    globals()[f"DNK_H2tanklink_{year}_charge_p0"] = tanklink_charge
    globals()[f"DNK_H2tanklink_{year}_discharge_p1"] = tanklink_discharge
    globals()[f"DNK_H2underlink_{year}_charge_p0"] = underlink_charge
    globals()[f"DNK_H2underlink_{year}_discharge_p1"] = underlink_discharge
    globals()[f"DNK_H2link_{year}_charge_p0"] = tanklink_charge + underlink_charge
    globals()[f"DNK_H2link_{year}_discharge_p1"] = tanklink_discharge + underlink_discharge

    batterylink_charge, batterylink_discharge = H2Link(n_year.stores_t.e["DK battery"],1,1)

    globals()[f"DNK_batterylink_{year}_charge_p0"] = batterylink_charge
    globals()[f"DNK_batterylink_{year}_discharge_p1"] = batterylink_discharge

    
#DEU
...

#ITA
...


In [ ]:
#################### Calculate the LCOS and UB of Storage, take DEU as example, same as DNK and ITA

In [ ]:
## First Part: 

# DEU Gas Demand

DEU_Gas_EleDemand_2020 = n_2020.links_t.p0["DE CCGT"] + n_2020.links_t.p0["DE OCGT"] + n_2020.links_t.p0["DE central gas CHP electric"]
DEU_Gas_EleDemand_2025 = ...

DEU_Gas_HeatDemand_2020 = n_2020.links_t.p0["DE central gas boiler"] + n_2020.links_t.p0["DE decentral gas boiler"]
DEU_Gas_HeatDemand_2025 = ...

DEU_Gas_Demand_2020 = DEU_Gas_EleDemand_2020 + DEU_Gas_HeatDemand_2020
DEU_Gas_Demand_2025 = ...

# DEU Gas Supply

DEU_Gas_Supply_2020 = - n_2020.links_t.p1["DE Sabatier"]
DEU_Gas_Supply_2025 = ...

# DEU H2 for Gas

DEU_H2_Consumption_2020 = n_2020.links_t.p0["DE Sabatier"]
DEU_H2_Consumption_2025 = ...


DEU_H2_Consumption_pnom_2020 = n_2020.links.p_nom_opt["DE Sabatier"]
DEU_H2_Consumption_pnom_2025 = ...

DEU_H2_Consumption_pnom = [DEU_H2_Consumption_pnom_2020,
                           DEU_H2_Consumption_pnom_2025 - DEU_H2_Consumption_pnom_2020,
                           DEU_H2_Consumption_pnom_2030 - DEU_H2_Consumption_pnom_2025,
                           DEU_H2_Consumption_pnom_2035 - DEU_H2_Consumption_pnom_2030,
                           DEU_H2_Consumption_pnom_2040 - DEU_H2_Consumption_pnom_2035,
                           DEU_H2_Consumption_pnom_2045 - DEU_H2_Consumption_pnom_2040 + DEU_H2_Consumption_pnom_2020,
                           DEU_H2_Consumption_pnom_2050 - DEU_H2_Consumption_pnom_2045 + DEU_H2_Consumption_pnom_2025 - DEU_H2_Consumption_pnom_2020]

DEU_H2_Consumption_pnom_ay = [DEU_H2_Consumption_pnom_2020,
                              DEU_H2_Consumption_pnom_2025,
                              DEU_H2_Consumption_pnom_2030,
                              DEU_H2_Consumption_pnom_2035,
                              DEU_H2_Consumption_pnom_2040,
                              DEU_H2_Consumption_pnom_2045,
                              DEU_H2_Consumption_pnom_2050]

DEU_H2_Consumption_capital_2020 = n_2020.links.capital_cost["DE Sabatier"]
DEU_H2_Consumption_capital_2025 = ...

DEU_H2_Consumption_capital = [DEU_H2_Consumption_capital_2020,
                              DEU_H2_Consumption_capital_2025,
                              DEU_H2_Consumption_capital_2030,
                              DEU_H2_Consumption_capital_2035,
                              DEU_H2_Consumption_capital_2040,
                              DEU_H2_Consumption_capital_2045,
                              DEU_H2_Consumption_capital_2050]

# H2 Sabatier (Discharge Link) Cost

DEU_H2_ConPnomCost_nb = [x * y for x, y in zip(DEU_H2_Consumption_pnom, DEU_H2_Consumption_capital)]

DEU_H2_ConPnomCost_ay = [x * y for x, y in zip(DEU_H2_Consumption_pnom_ay, DEU_H2_Consumption_capital)]

DEU_H2_ConPnom_accum_allcost = [DEU_H2_ConPnomCost_nb[0],
                                DEU_H2_ConPnomCost_nb[1] + DEU_H2_ConPnomCost_nb[0],
                                DEU_H2_ConPnomCost_nb[2] + DEU_H2_ConPnomCost_nb[1] + DEU_H2_ConPnomCost_nb[0],
                                DEU_H2_ConPnomCost_nb[3] + DEU_H2_ConPnomCost_nb[2] + DEU_H2_ConPnomCost_nb[1] + DEU_H2_ConPnomCost_nb[0],
                                DEU_H2_ConPnomCost_nb[4] + DEU_H2_ConPnomCost_nb[3] + DEU_H2_ConPnomCost_nb[2] + DEU_H2_ConPnomCost_nb[1] + DEU_H2_ConPnomCost_nb[0],
                                DEU_H2_ConPnomCost_nb[5] + DEU_H2_ConPnomCost_nb[4] + DEU_H2_ConPnomCost_nb[3] + DEU_H2_ConPnomCost_nb[2] + DEU_H2_ConPnomCost_nb[1],
                                DEU_H2_ConPnomCost_nb[6] + DEU_H2_ConPnomCost_nb[5] + DEU_H2_ConPnomCost_nb[4] + DEU_H2_ConPnomCost_nb[3] + DEU_H2_ConPnomCost_nb[2]]


# DEU H2 Marginal Price

DEU_H2_MarginalPrice_2020 = n_2020.buses_t.marginal_price["DE H2"]
DEU_H2_MarginalPrice_2025 = ...

# Total H2 sale Revenue
DEU_H2_SaleRevenue_all = [(x * y).sum() for x, y in zip(DEU_H2_Consumption_amount, DEU_H2_MarginalPrice)]

## Second Part: charging and discharging (Link) cost + Revenue

DEU_H2_Production_2020 = n_2020.links_t.p0["DE H2 Electrolysis"]
DEU_H2_Production_2025 = ...

# Total H2 Chaging Cost
DEU_H2_ChargingCost_all = [(x * y["DE"]).sum() for x, y in zip(DEU_H2_Production_amount_all, DEU_EleP)]

DEU_H2_Production_pnom_2020 = n_2020.links.p_nom_opt["DE H2 Electrolysis"]
DEU_H2_Production_pnom_2025 = ...

DEU_H2_Production_pnom = ...
DEU_H2_Production_pnom_ay = [DEU_H2_Production_pnom_2020,
                             ...]

DEU_H2_Production_capital_2020 = n_2020.links.capital_cost["DE H2 Electrolysis"]
DEU_H2_Production_capital_2025 = ...

# Total H2 Chaging Link Cost

DEU_H2_ProPnomCost_nb = [x * y for x, y in zip(DEU_H2_Production_pnom, DEU_H2_Production_capital)]

DEU_H2_ProPnomCost_ay = [x * y for x, y in zip(DEU_H2_Production_pnom_ay, DEU_H2_Production_capital)]

DEU_H2_ProPnom_accum_allcost = [DEU_H2_ProPnomCost_nb[0],
                                DEU_H2_ProPnomCost_nb[1] + DEU_H2_ProPnomCost_nb[0],
                                DEU_H2_ProPnomCost_nb[2] + DEU_H2_ProPnomCost_nb[1] + DEU_H2_ProPnomCost_nb[0],
                                DEU_H2_ProPnomCost_nb[3] + DEU_H2_ProPnomCost_nb[2] + DEU_H2_ProPnomCost_nb[1] + DEU_H2_ProPnomCost_nb[0],
                                DEU_H2_ProPnomCost_nb[4] + DEU_H2_ProPnomCost_nb[3] + DEU_H2_ProPnomCost_nb[2] + DEU_H2_ProPnomCost_nb[1] + DEU_H2_ProPnomCost_nb[0],
                                DEU_H2_ProPnomCost_nb[5] + DEU_H2_ProPnomCost_nb[4] + DEU_H2_ProPnomCost_nb[3] + DEU_H2_ProPnomCost_nb[2] + DEU_H2_ProPnomCost_nb[1],
                                DEU_H2_ProPnomCost_nb[6] + DEU_H2_ProPnomCost_nb[5] + DEU_H2_ProPnomCost_nb[4] + DEU_H2_ProPnomCost_nb[3] + DEU_H2_ProPnomCost_nb[2]]

# directly electricity, fuel cell

DEU_H2_FuelCell_2020 = - n_2020.links_t.p1["DE H2 Fuel Cell"]
DEU_H2_FuelCell_2025 = ...

# Directly Electricity Revenue

DEU_H2_FuelCell_Revenue = [(x * y["DE"]).sum() for x, y in zip(DEU_H2_FuelCell_amount, DEU_EleP)]

DEU_H2_FuelCell_pnom_2020 = n_2020.links.p_nom_opt["DE H2 Fuel Cell"]
DEU_H2_FuelCell_pnom_2025 = ...

DEU_H2_FuelCell_pnom = ...
DEU_H2_FuelCell_pnom_ay = [DEU_H2_FuelCell_pnom_2020,
                           ...]

DEU_H2_FuelCell_capital_2020 = n_2020.links.capital_cost["DE H2 Fuel Cell"]
DEU_H2_FuelCell_capital_2025 = ...

# Fuel Cell Discharging Link Cost
                                
DEU_H2_FCCost_nb = [x * y for x, y in zip(DEU_H2_FuelCell_pnom, DEU_H2_FuelCell_capital)]
DEU_H2_FCCost_ay = [x * y for x, y in zip(DEU_H2_FuelCell_pnom_ay, DEU_H2_FuelCell_capital)]

DEU_H2_FCPnom_accum_allcost = [DEU_H2_FCCost_nb[0],
                               DEU_H2_FCCost_nb[1] + DEU_H2_FCCost_nb[0],
                               DEU_H2_FCCost_nb[2] + DEU_H2_FCCost_nb[1],
                               DEU_H2_FCCost_nb[3] + DEU_H2_FCCost_nb[2],
                               DEU_H2_FCCost_nb[4] + DEU_H2_FCCost_nb[3],
                               DEU_H2_FCCost_nb[5] + DEU_H2_FCCost_nb[4],
                               DEU_H2_FCCost_nb[6] + DEU_H2_FCCost_nb[5]]

DEU_e_forsa = [x - y / FuelCell_efficiency for x, y in zip(DEU_H2_e_Discharge_amount, DEU_H2_FuelCell_amount)]

DEU_e_forsa_revenue = [max(0, (x * y).sum()) for x, y in zip(DEU_e_forsa, DEU_H2_MarginalPrice)]
    
DEU_p_forsa = [x * Electrolysis_efficiency - y for x, y in zip(DEU_H2_Production_amount_all, DEU_H2_e_Charge_amount)]

DEU_p_forsa_revenue = [(x * y).sum() for x, y in zip(DEU_p_forsa, DEU_H2_MarginalPrice)]

DEU_broad_storage_pnom = [min(1, (x.sum() / (y.sum()*Electrolysis_efficiency))) * z for x, y, z in zip(DEU_p_forsa, DEU_H2_Production_amount_all, DEU_H2_Production_pnom_ay)]


# calculate the ratio

years = [2020, 2025, 2030, 2035, 2040, 2045, 2050]

DEU_e_forele_list = []
DEU_p_forele_list = []
DEU_e_forheat_list = []
DEU_p_forheat_list = []
DEU_Ind_ele_list = []

for year in years:
    year_index = years.index(year)
    
    if sum(DEU_Gas_Demand_amount[year_index]) > sum(DEU_Gas_Supply_amount[year_index]):
        DEU_Ind_ele = (sum(DEU_Gas_EleDemand_amount[year_index]) / sum(DEU_Gas_Demand_amount[year_index])) * sum(DEU_H2_Consumption_amount[year_index])
        DEU_e_forele = (sum(DEU_Gas_EleDemand_amount[year_index]) / sum(DEU_Gas_Demand_amount[year_index])) * sum(DEU_e_forsa[year_index])
        DEU_p_forele = (sum(DEU_Gas_EleDemand_amount[year_index]) / sum(DEU_Gas_Demand_amount[year_index])) * sum(DEU_p_forsa[year_index])
        
        DEU_e_forheat = (1 - sum(DEU_Gas_EleDemand_amount[year_index]) / sum(DEU_Gas_Demand_amount[year_index])) * sum(DEU_e_forsa[year_index])
        DEU_p_forheat = (1 - sum(DEU_Gas_EleDemand_amount[year_index]) / sum(DEU_Gas_Demand_amount[year_index])) * sum(DEU_p_forsa[year_index])
        
        if DEU_e_forele < 0:
            DEU_p_forele = DEU_p_forele + DEU_e_forele
            DEU_e_forele = 0
            DEU_e_forheat = 0

    else:
        DEU_Ind_ele = gas_electricity_ratios[year] * sum(DEU_H2_Consumption_amount[year_index])
        DEU_e_forele = gas_electricity_ratios[year] * sum(DEU_e_forsa[year_index])
        DEU_p_forele = gas_electricity_ratios[year] * sum(DEU_p_forsa[year_index])
        
        DEU_e_forheat = (1 - gas_electricity_ratios[year]) * sum(DEU_e_forsa[year_index])
        DEU_p_forheat = (1 - gas_electricity_ratios[year]) * sum(DEU_p_forsa[year_index])
        
        if DEU_e_forele < 0:
            DEU_p_forele = DEU_p_forele + DEU_e_forele
            DEU_e_forele = 0
            DEU_e_forheat = 0
        
    DEU_Ind_ele_list.append(DEU_Ind_ele)
    DEU_e_forele_list.append(DEU_e_forele)
    DEU_p_forele_list.append(DEU_p_forele)
    DEU_e_forheat_list.append(DEU_e_forheat)
    DEU_p_forheat_list.append(DEU_p_forheat)

DEU_broad_heat_pnom = [min(1, (x / y.sum())) * z for x, y, z in zip(DEU_p_forheat_list, DEU_p_forsa, DEU_broad_storage_pnom)]
DEU_broad_ele_pnom = [min(1, (x / y.sum())) * z for x, y, z in zip(DEU_p_forele_list, DEU_p_forsa, DEU_broad_storage_pnom)]

################### Charging Cost ###################

# 1. fuel cell
DEU_H2_CC_fc = [(((x.sum()/H2_Around)/(y.sum())))*z for x, y, z in zip(DEU_H2_FuelCell_amount, DEU_H2_Production_amount_all, DEU_H2_ChargingCost_all)]

# 2. storage for electricity through sabatier
DEU_H2_CC_ee = [((x/Electrolysis_efficiency)/(y.sum()))*z for x, y, z in zip(DEU_e_forele_list, DEU_H2_Production_amount_all, DEU_H2_ChargingCost_all)]

# 3. storage for heating through sabatier
DEU_H2_CC_eh = [((x/Electrolysis_efficiency)/(y.sum()))*z for x, y, z in zip(DEU_e_forheat_list, DEU_H2_Production_amount_all, DEU_H2_ChargingCost_all)]

# 4. H2 production directly for electricity through sabatier
DEU_H2_CC_pe = [((x/Electrolysis_efficiency)/(y.sum()))*z for x, y, z in zip(DEU_p_forele_list, DEU_H2_Production_amount_all, DEU_H2_ChargingCost_all)]

# 5. H2 production directly for heating through sabatier
DEU_H2_CC_ph = [((x/Electrolysis_efficiency)/(y.sum()))*z for x, y, z in zip(DEU_p_forheat_list, DEU_H2_Production_amount_all, DEU_H2_ChargingCost_all)]

################### Discharging Cost (Revenue) ###################

# 1. fuel cell
DEU_H2_DC_fc = DEU_H2_FuelCell_Revenue

# 2. storage for electricity through sabatier
DEU_H2_DC_ee = [(x/abs((y.sum())))*z for x, y, z in zip(DEU_e_forele_list, DEU_e_forsa, DEU_e_forsa_revenue)]

# 3. storage for heating through sabatier
DEU_H2_DC_eh = [(x/abs((y.sum())))*z for x, y, z in zip(DEU_e_forheat_list, DEU_e_forsa, DEU_e_forsa_revenue)]

# 4. H2 production directly for electricity through sabatier
DEU_H2_DC_pe = [(x/abs((y.sum())))*z for x, y, z in zip(DEU_p_forele_list, DEU_p_forsa, DEU_p_forsa_revenue)]

# 5. H2 production directly for heating through sabatier
DEU_H2_DC_ph = [(x/abs((y.sum())))*z for x, y, z in zip(DEU_p_forheat_list, DEU_p_forsa, DEU_p_forsa_revenue)]

################### Charging Link Cost (with accumlated) ###################

# 1. fuel cell
DEU_H2_CLC_fc_ac = [(((x.sum()/H2_Around)/(y.sum())))*z for x, y, z in zip(DEU_H2_FuelCell_amount, DEU_H2_Production_amount_all, DEU_H2_ProPnom_accum_allcost)]

# 2. storage for electricity through sabatier
DEU_H2_CLC_ee_ac = [((x/Electrolysis_efficiency)/(y.sum()))*z for x, y, z in zip(DEU_e_forele_list, DEU_H2_Production_amount_all, DEU_H2_ProPnom_accum_allcost)]

# 3. storage for heating through sabatier
DEU_H2_CLC_eh_ac = [((x/Electrolysis_efficiency)/(y.sum()))*z for x, y, z in zip(DEU_e_forheat_list, DEU_H2_Production_amount_all, DEU_H2_ProPnom_accum_allcost)]

# 4. H2 production directly for electricity through sabatier
DEU_H2_CLC_pe_ac = [((x/Electrolysis_efficiency)/(y.sum()))*z for x, y, z in zip(DEU_p_forele_list, DEU_H2_Production_amount_all, DEU_H2_ProPnom_accum_allcost)]

# 5. H2 production directly for heating through sabatier
DEU_H2_CLC_ph_ac = [((x/Electrolysis_efficiency)/(y.sum()))*z for x, y, z in zip(DEU_p_forheat_list, DEU_H2_Production_amount_all, DEU_H2_ProPnom_accum_allcost)]

################### Charging Link Cost (without accumlated) ###################

# 1. fuel cell
DEU_H2_CLC_fc_ay = [(((x.sum()/H2_Around)/(y.sum())))*z for x, y, z in zip(DEU_H2_FuelCell_amount, DEU_H2_Production_amount_all, DEU_H2_ProPnomCost_ay)]

# 2. storage for electricity through sabatier
DEU_H2_CLC_ee_ay = [((x/Electrolysis_efficiency)/(y.sum()))*z for x, y, z in zip(DEU_e_forele_list, DEU_H2_Production_amount_all, DEU_H2_ProPnomCost_ay)]

# 3. storage for heating through sabatier
DEU_H2_CLC_eh_ay = [((x/Electrolysis_efficiency)/(y.sum()))*z for x, y, z in zip(DEU_e_forheat_list, DEU_H2_Production_amount_all, DEU_H2_ProPnomCost_ay)]

# 4. H2 production directly for electricity through sabatier
DEU_H2_CLC_pe_ay = [((x/Electrolysis_efficiency)/(y.sum()))*z for x, y, z in zip(DEU_p_forele_list, DEU_H2_Production_amount_all, DEU_H2_ProPnomCost_ay)]

# 5. H2 production directly for heating through sabatier
DEU_H2_CLC_ph_ay = [((x/Electrolysis_efficiency)/(y.sum()))*z for x, y, z in zip(DEU_p_forheat_list, DEU_H2_Production_amount_all, DEU_H2_ProPnomCost_ay)]

################### Discharging Link Cost (with accumalated, without sabatier) ###################

# 1. fuel cell
DEU_H2_DLC_fc_ac = DEU_H2_FCPnom_accum_allcost

# else
DEU_H2_DLC_ee_ac = [0, 0, 0, 0, 0, 0, 0]
DEU_H2_DLC_eh_ac = [0, 0, 0, 0, 0, 0, 0]
DEU_H2_DLC_pe_ac = [0, 0, 0, 0, 0, 0, 0]
DEU_H2_DLC_ph_ac = [0, 0, 0, 0, 0, 0, 0]

################### Discharging Link Cost (without accumalated, without sabatier) ###################

# 1. fuel cell
DEU_H2_DLC_fc_ay = DEU_H2_FCCost_ay

# else
DEU_H2_DLC_ee_ay = [0, 0, 0, 0, 0, 0, 0]
DEU_H2_DLC_eh_ay = [0, 0, 0, 0, 0, 0, 0]
DEU_H2_DLC_pe_ay = [0, 0, 0, 0, 0, 0, 0]
DEU_H2_DLC_ph_ay = [0, 0, 0, 0, 0, 0, 0]

## Third Part: Total and divided enom of storage

# DEU H2 underground storage, enom & capacity cost, 100 years

DEU_H2_Underground_2020 = n_2020.stores_t.e["DE H2 Store underground"]
DEU_H2_Underground_2025 = ...

DEU_H2_Underground_enom_2020 = n_2020.stores.e_nom_opt["DE H2 Store underground"]
DEU_H2_Underground_enom_2025 = ...

DEU_H2_Underground_enom = [DEU_H2_Underground_enom_2020,
                           ...]

DEU_H2_Underground_enom_ay = [DEU_H2_Underground_enom_2020,
                              ...]

DEU_H2_Underground_capital_2020 = n_2020.stores.capital_cost["DE H2 Store underground"]
DEU_H2_Underground_capital_2025 = ...

DEU_H2_UGCost_nb = [x * y for x, y in zip(DEU_H2_Underground_enom, DEU_H2_Underground_capital)]
DEU_H2_UGCost_ay = [x * y for x, y in zip(DEU_H2_Underground_enom_ay, DEU_H2_Underground_capital)]
            
DEU_H2_UGEnom_accum_allcost = [DEU_H2_UGCost_nb[0],
                               ...]

# DEU H2 Tank storage, enom & capacity cost, 25 years

DEU_H2_Tank_2020 = n_2020.stores_t.e["DE H2 Store tank"]
DEU_H2_Tank_2025 = ...

DEU_H2_Tank_enom_2020 = n_2020.stores.e_nom_opt["DE H2 Store tank"]
DEU_H2_Tank_enom_2025 = ...

DEU_H2_Tank_capital_2020 = n_2020.stores.capital_cost["DE H2 Store tank"]
DEU_H2_Tank_capital_2025 = ...

DEU_H2_TCost_nb = [x * y for x, y in zip(DEU_H2_Tank_enom, DEU_H2_Tank_capital)]
DEU_H2_TCost_ay = [x * y for x, y in zip(DEU_H2_Tank_enom_ay, DEU_H2_Tank_capital)]
            
DEU_H2_TEnom_accum_allcost = [DEU_H2_TCost_nb[0],
                              DEU_H2_TCost_nb[1] + DEU_H2_TCost_nb[0],
                              ...]

# Combine tank and underground
DEU_H2All_enom = [x + y for x, y in zip(DEU_H2_Tank_enom, DEU_H2_Underground_enom)]
DEU_H2All_enom_ay = [x + y for x, y in zip(DEU_H2_Tank_enom_ay, DEU_H2_Underground_enom_ay)]

DEU_H2All_ECost = [x + y for x, y in zip(DEU_H2_TEnom_accum_allcost, DEU_H2_UGEnom_accum_allcost)]
DEU_H2All_ECost_ay = [x + y for x, y in zip(DEU_H2_TCost_ay, DEU_H2_UGCost_ay)]

################### Storage Cost (with accumlated) ###################

# 1. fuel cell
DEU_H2_SC_fc_ac = [
    (1 if ((x.sum()/FuelCell_efficiency)/(y.sum())) > 1 else ((x.sum()/FuelCell_efficiency)/(y.sum())))*z 
    for x, y, z in zip(DEU_H2_FuelCell_amount, DEU_H2_e_Discharge_amount, DEU_H2All_ECost)
]

# 2. storage for electricity through sabatier
DEU_H2_SC_ee_ac = [
    (1 if (x/(y.sum())) > 1 else (x/(y.sum())))*z 
    for x, y, z in zip(DEU_e_forele_list, DEU_H2_e_Discharge_amount, DEU_H2All_ECost)
]

# 3. storage for heating through sabatier
DEU_H2_SC_eh_ac = [
    (1 if (x/(y.sum())) > 1 else (x/(y.sum())))*z 
    for x, y, z in zip(DEU_e_forheat_list, DEU_H2_e_Discharge_amount, DEU_H2All_ECost)
]

################### Storage Cost (without accumlated) ###################

# 1. fuel cell
DEU_H2_SC_fc_ay = [
    (1 if ((x.sum()/FuelCell_efficiency)/(y.sum())) > 1 else ((x.sum()/FuelCell_efficiency)/(y.sum())))*z 
    for x, y, z in zip(DEU_H2_FuelCell_amount, DEU_H2_e_Discharge_amount, DEU_H2All_ECost_ay)
]

# 2. storage for electricity through sabatier
DEU_H2_SC_ee_ay = [
    (1 if (x/(y.sum())) > 1 else (x/(y.sum())))*z 
    for x, y, z in zip(DEU_e_forele_list, DEU_H2_e_Discharge_amount, DEU_H2All_ECost_ay)
]

# 3. storage for heating through sabatier
DEU_H2_SC_eh_ay = [
    (1 if (x/(y.sum())) > 1 else (x/(y.sum())))*z 
    for x, y, z in zip(DEU_e_forheat_list, DEU_H2_e_Discharge_amount, DEU_H2All_ECost_ay)
]

################### Storage Capacity Distribute ###################

# 1. fuel cell
DEU_H2_E_fc = [
    (1 if ((x.sum()/FuelCell_efficiency)/(y.sum())) > 1 else ((x.sum()/FuelCell_efficiency)/(y.sum())))*z 
    for x, y, z in zip(DEU_H2_FuelCell_amount, DEU_H2_e_Discharge_amount, DEU_H2All_enom_ay)
]

# 2. storage for electricity through sabatier
DEU_H2_E_ee = [
    (1 if (x/(y.sum())) > 1 else (x/(y.sum())))*z 
    for x, y, z in zip(DEU_e_forele_list, DEU_H2_e_Discharge_amount, DEU_H2All_enom_ay)
]

# 3. storage for heating through sabatier
DEU_H2_E_eh = [
    (1 if (x/(y.sum())) > 1 else (x/(y.sum())))*z 
    for x, y, z in zip(DEU_e_forheat_list, DEU_H2_e_Discharge_amount, DEU_H2All_enom_ay)
]

### Fourth Part: H2 Parameter Collect (all use accumlated value, without Sabatier Investment)

## here we will make 7 group to show the H2 benefits and LOCS
## attention all the unit will be the unit of H2

# Group 1. Just Fuel Cell

DEU_H201_Invest = [x + y + z for x, y, z in zip(DEU_H2_SC_fc_ac, DEU_H2_CLC_fc_ac, DEU_H2_DLC_fc_ac)]
DEU_H201_Benefit = [max(0, x - y) for x, y in zip(DEU_H2_DC_fc, DEU_H2_CC_fc)]

DEU_H201_DA = [x.sum()/FuelCell_efficiency for x in DEU_H2_FuelCell_amount]

DEU_H201_UnitInvest = [x / y for x, y in zip(DEU_H201_Invest, DEU_H201_DA)]
DEU_H201_UnitBenefit = [x / y for x, y in zip(DEU_H201_Benefit, DEU_H201_DA)]

# Group 2. Just electricity from storage to sabatier

DEU_H202_Invest = [x + y + z for x, y, z in zip(DEU_H2_SC_ee_ac, DEU_H2_CLC_ee_ac, DEU_H2_DLC_ee_ac)]
DEU_H202_Benefit = [max(0, x - y) for x, y in zip(DEU_H2_DC_ee, DEU_H2_CC_ee)]

DEU_H202_DA = DEU_e_forele_list

DEU_H202_UnitInvest = [0 if y == 0 else x / y for x, y in zip(DEU_H202_Invest, DEU_H202_DA)]
DEU_H202_UnitBenefit = [0 if y == 0 else x / y for x, y in zip(DEU_H202_Benefit, DEU_H202_DA)]

# Group 3. Just electricity from H2 production directly to sabatier

DEU_H203_Invest = [x + y for x, y in zip(DEU_H2_CLC_pe_ac, DEU_H2_DLC_pe_ac)]
DEU_H203_Benefit = [max(0, x - y) for x, y in zip(DEU_H2_DC_pe, DEU_H2_CC_pe)]

DEU_H203_DA = DEU_p_forele_list

DEU_H203_UnitInvest = [0 if y == 0 else x / y for x, y in zip(DEU_H203_Invest, DEU_H203_DA)]
DEU_H203_UnitBenefit = [0 if y == 0 else x / y for x, y in zip(DEU_H203_Benefit, DEU_H203_DA)]

# Group 4, all electricity from H2 storage (Group 1 + Group 2)

DEU_H204_Invest = [x + y for x, y in zip(DEU_H201_Invest, DEU_H202_Invest)]
DEU_H204_Benefit = [max(0, x + y) for x, y in zip(DEU_H201_Benefit, DEU_H202_Benefit)]

DEU_H204_DA = [x + y for x, y in zip(DEU_H201_DA, DEU_H202_DA)]

DEU_H204_UnitInvest = [0 if y == 0 else x / y for x, y in zip(DEU_H204_Invest, DEU_H204_DA)]
DEU_H204_UnitBenefit = [0 if y == 0 else x / y for x, y in zip(DEU_H204_Benefit, DEU_H204_DA)]

# Group 5, all electricity from H2 to sabatier (Group 1 + Group 2 + Group 3)

DEU_H205_Invest = [x + y for x, y in zip(DEU_H203_Invest, DEU_H204_Invest)]
DEU_H205_Benefit = [max(0, x + y) for x, y in zip(DEU_H203_Benefit, DEU_H204_Benefit)]

DEU_H205_DA = [x + y for x, y in zip(DEU_H203_DA, DEU_H204_DA)]

DEU_H205_UnitInvest = [0 if y == 0 else x / y for x, y in zip(DEU_H205_Invest, DEU_H205_DA)]
DEU_H205_UnitBenefit = [0 if y == 0 else x / y for x, y in zip(DEU_H205_Benefit, DEU_H205_DA)]

# Group 6, all storage performance (Group 1 + Group 2 + storage_for_heating)

DEU_H206_Invest = [x + y + z + d for x, y, z, d in zip(DEU_H204_Invest, DEU_H2_SC_eh_ac, DEU_H2_CLC_eh_ac, DEU_H2_DLC_eh_ac)]
DEU_H206_Benefit = [max(0, x + y - z) for x, y, z in zip(DEU_H204_Benefit, DEU_H2_DC_eh, DEU_H2_CC_eh)]

DEU_H206_DA = [x + y for x, y in zip(DEU_H204_DA, DEU_e_forheat_list)]

DEU_H206_UnitInvest = [0 if y == 0 else x / y for x, y in zip(DEU_H206_Invest, DEU_H206_DA)]
DEU_H206_UnitBenefit = [0 if y == 0 else x / y for x, y in zip(DEU_H206_Benefit, DEU_H206_DA)]

# Group 7, all H2 performance

DEU_H207_Invest = [x + y + z + d + a for x, y, z, d, a in zip(DEU_H206_Invest, DEU_H2_CLC_ph_ac, DEU_H2_DLC_ph_ac, DEU_H2_CLC_pe_ac, DEU_H2_DLC_pe_ac)]
DEU_H207_Benefit = [max(0, x + y - z + a - b) for x, y, z, a, b in zip(DEU_H206_Benefit, DEU_H2_DC_pe, DEU_H2_CC_pe, DEU_H2_DC_ph, DEU_H2_CC_ph)]

DEU_H207_DA = [x + y + z for x, y, z in zip(DEU_H206_DA, DEU_p_forele_list, DEU_p_forheat_list)]

DEU_H207_UnitInvest = [0 if y == 0 else x / y for x, y in zip(DEU_H207_Invest, DEU_H207_DA)]
DEU_H207_UnitBenefit = [0 if y == 0 else x / y for x, y in zip(DEU_H207_Benefit, DEU_H207_DA)]

### Fifth Part: H2 Parameter Collect (all use not accumlated value, without Sabatier Investment)

## here we will make 7 group to show the H2 benefits and LOCS
## attention all the unit will be the unit of H2

# Group 8. Just Fuel Cell

DEU_H208_Invest = [x + y + z for x, y, z in zip(DEU_H2_SC_fc_ay, DEU_H2_CLC_fc_ay, DEU_H2_DLC_fc_ay)]

DEU_H208_DA = DEU_H201_DA

DEU_H208_UnitInvest = [x / y for x, y in zip(DEU_H208_Invest, DEU_H208_DA)]

# Group 9. Just electricity from storage to sabatier

DEU_H209_Invest = [x + y + z for x, y, z in zip(DEU_H2_SC_ee_ay, DEU_H2_CLC_ee_ay, DEU_H2_DLC_ee_ay)]

DEU_H209_DA = DEU_H202_DA

DEU_H209_UnitInvest = [0 if y == 0 else x / y for x, y in zip(DEU_H209_Invest, DEU_H209_DA)]

# Group 10. Just electricity from H2 production directly to sabatier

DEU_H210_Invest = [x + y for x, y in zip(DEU_H2_CLC_pe_ay, DEU_H2_DLC_pe_ay)]

DEU_H210_DA = DEU_H203_DA

DEU_H210_UnitInvest = [0 if y == 0 else x / y for x, y in zip(DEU_H210_Invest, DEU_H210_DA)]

# Group 11, all electricity from H2 storage (Group 1 + Group 2)

DEU_H211_Invest = [x + y for x, y in zip(DEU_H208_Invest, DEU_H209_Invest)]

DEU_H211_DA = DEU_H204_DA

DEU_H211_UnitInvest = [0 if y == 0 else x / y for x, y in zip(DEU_H211_Invest, DEU_H211_DA)]

# Group 12, all electricity from H2 to sabatier (Group 1 + Group 2 + Group 3)

DEU_H212_Invest = [x + y for x, y in zip(DEU_H210_Invest, DEU_H211_Invest)]

DEU_H212_DA = DEU_H205_DA

DEU_H212_UnitInvest = [0 if y == 0 else x / y for x, y in zip(DEU_H212_Invest, DEU_H212_DA)]

# Group 13, all storage performance (Group 1 + Group 2 + storage_for_heating)

DEU_H213_Invest = [x + y + z + d for x, y, z, d in zip(DEU_H211_Invest, DEU_H2_SC_eh_ay, DEU_H2_CLC_eh_ay, DEU_H2_DLC_eh_ay)]

DEU_H213_DA = DEU_H206_DA

DEU_H213_UnitInvest = [0 if y == 0 else x / y for x, y in zip(DEU_H213_Invest, DEU_H213_DA)]

# Group 14, all H2 performance

DEU_H214_Invest = [x + y + z + d + a for x, y, z, d, a in zip(DEU_H213_Invest, DEU_H2_CLC_ph_ay, DEU_H2_DLC_ph_ay, DEU_H2_CLC_pe_ay, DEU_H2_DLC_pe_ay)]

DEU_H214_DA = DEU_H207_DA

DEU_H214_UnitInvest = [0 if y == 0 else x / y for x, y in zip(DEU_H214_Invest, DEU_H214_DA)]


# Group 15, broad H2 storage, accumulated

DEU_H215_Invest = [x + y + z for x, y, z in zip(DEU_H203_Invest, DEU_H2_CLC_ph_ac, DEU_H2_DLC_ph_ac)]
DEU_H215_Benefit = [max(0, x + y - z) for x, y, z in zip(DEU_H203_Benefit, DEU_H2_DC_ph, DEU_H2_CC_ph)]

DEU_H215_DA = [x + y for x, y in zip(DEU_p_forele_list, DEU_p_forheat_list)]

DEU_H215_UnitInvest = [0 if y == 0 else x / y for x, y in zip(DEU_H215_Invest, DEU_H215_DA)]
DEU_H215_UnitBenefit = [0 if y == 0 else x / y for x, y in zip(DEU_H215_Benefit, DEU_H215_DA)]

# Group 16, broad H2 storage, without accumulated

DEU_H216_Invest = [x + y + z for x, y, z in zip(DEU_H210_Invest, DEU_H2_CLC_ph_ay, DEU_H2_DLC_ph_ay)]

DEU_H216_DA = DEU_H215_DA

DEU_H216_UnitInvest = [0 if y == 0 else x / y for x, y in zip(DEU_H216_Invest, DEU_H216_DA)]

# Group 17, storage for heating, accumulated

DEU_H217_Invest = [x + y + z for x, y, z in zip(DEU_H2_SC_eh_ac, DEU_H2_CLC_eh_ac, DEU_H2_DLC_eh_ac)]
DEU_H217_Benefit = [max(0, x - y) for x, y in zip(DEU_H2_DC_eh, DEU_H2_CC_eh)]

DEU_H217_DA = DEU_e_forheat_list

DEU_H217_UnitInvest = [0 if y == 0 else x / y for x, y in zip(DEU_H217_Invest, DEU_H217_DA)]
DEU_H217_UnitBenefit = [0 if y == 0 else x / y for x, y in zip(DEU_H217_Benefit, DEU_H217_DA)]

# Group 18, storage for heating, without accumulated

DEU_H218_Invest = [x + y + z for x, y, z in zip(DEU_H2_SC_eh_ay, DEU_H2_CLC_eh_ay, DEU_H2_DLC_eh_ay)]

DEU_H218_DA = DEU_H217_DA

DEU_H218_UnitInvest = [0 if y == 0 else x / y for x, y in zip(DEU_H218_Invest, DEU_H217_DA)]

# Group 19, production for heating, accumulated

DEU_H219_Invest = [x + y for x, y in zip(DEU_H2_CLC_ph_ac, DEU_H2_DLC_ph_ac)]
DEU_H219_Benefit = [max(0, x - y) for x, y in zip(DEU_H2_DC_ph, DEU_H2_CC_ph)]

DEU_H219_DA = DEU_p_forheat_list

DEU_H219_UnitInvest = [0 if y == 0 else x / y for x, y in zip(DEU_H219_Invest, DEU_H219_DA)]
DEU_H219_UnitBenefit = [0 if y == 0 else x / y for x, y in zip(DEU_H219_Benefit, DEU_H219_DA)]

# Group 20, production for heating, without accumulated

DEU_H220_Invest = [x + y for x, y in zip(DEU_H2_CLC_ph_ay, DEU_H2_DLC_ph_ay)]

DEU_H220_DA = DEU_H219_DA

DEU_H220_UnitInvest = [0 if y == 0 else x / y for x, y in zip(DEU_H220_Invest, DEU_H220_DA)]


# For battery

# Constant

charge_efficiency = [0.95, 0.955, 0.96, 0.96, 0.96, 0.96, 0.96]
discharge_efficiency = [0.95, 0.955, 0.96, 0.96, 0.96, 0.96, 0.96]
battery_around = [x * y for x, y in zip(charge_efficiency, discharge_efficiency)]

# charge

DEU_Battery_Charge_2020 = n_2020.links_t.p0["DE battery charger"]
DEU_Battery_Charge_2025 = ...

DEU_Battery_Charge_Cost = [(x * y["DE"]).sum() for x, y in zip(DEU_Battery_Charge_amount, DEU_EleP)]

# Charge_pnom and cost

DEU_Battery_Charge_pnom_2020 = n_2020.links.p_nom_opt["DE battery charger"]
DEU_Battery_Charge_pnom_2025 = ...

DEU_Battery_Charge_capital_2020 = n_2020.links.capital_cost["DE battery charger"]
DEU_Battery_Charge_capital_2025 = ...

DEU_Battery_CPnomcost_nb = [x * y for x, y in zip(DEU_Battery_Charge_pnom, DEU_Battery_Charge_capital)]
DEU_Battery_CPnomcost_ay = [x * y for x, y in zip(DEU_Battery_Charge_pnom_ay, DEU_Battery_Charge_capital)]

DEU_Battery_CPnom_accum_allcost = [DEU_Battery_CPnomcost_nb[0],
                                   DEU_Battery_CPnomcost_nb[1] + DEU_Battery_CPnomcost_nb[0],
                                   ...]

# Discharge

DEU_Battery_Discharge_2020 = - n_2020.links_t.p1["DE battery discharger"]
DEU_Battery_Discharge_2025 = - ...

DEU_Battery_Discharge_Cost = [(x * y["DE"]).sum() for x, y in zip(DEU_Battery_Discharge_amount, DEU_EleP)]

# DEU Battery Marginal Price

DEU_Battery_MarginalPrice_2020 = n_2020.buses_t.marginal_price["DE battery"]
DEU_Battery_MarginalPrice_2025 = ...

# Total Battery sale Revenue
DEU_Battery_SaleRevenue_all = [(x * y).sum()/z for x, y, z in zip(DEU_Battery_Discharge_amount, DEU_Battery_MarginalPrice, discharge_efficiency)]

# enom and cost

DEU_Battery_enom_2020 = n_2020.stores.e_nom_opt["DE battery"]
DEU_Battery_enom_2025 = ...

DEU_Battery_e_capital_2020 = n_2020.stores.capital_cost["DE battery"]
DEU_Battery_e_capital_2025 = ...

DEU_Battery_ECost_nb = [x * y for x, y in zip(DEU_Battery_enom, DEU_Battery_e_capital)]
DEU_Battery_ECost_ay = [x * y for x, y in zip(DEU_Battery_enom_ay, DEU_Battery_e_capital)]

DEU_Battery_Enom_accum_allcost = [DEU_Battery_ECost_nb[0],
                                  DEU_Battery_ECost_nb[1] + DEU_Battery_ECost_nb[0],
                                  ...]

# Calculate invest and benefits

DEU_Battery_Invest_ac = [x + y for x, y in zip(DEU_Battery_Enom_accum_allcost, DEU_Battery_CPnom_accum_allcost)]
DEU_Battery_Invest = [x + y for x, y in zip(DEU_Battery_ECost_ay, DEU_Battery_CPnomcost_ay)]
DEU_Battery_Benefit = [max(0, (x - y)) for x, y in zip(DEU_Battery_Discharge_Cost, DEU_Battery_Charge_Cost)]

DEU_Battery_UnitInvest_ac = [x / (y.sum()/z) for x, y, z in zip(DEU_Battery_Invest_ac, DEU_Battery_Discharge_amount, discharge_efficiency)]
DEU_Battery_UnitInvest = [x / (y.sum()/z) for x, y, z in zip(DEU_Battery_Invest, DEU_Battery_Discharge_amount, discharge_efficiency)]
DEU_Battery_UnitBenefit = [x / (y.sum()/z) for x, y, z in zip(DEU_Battery_Benefit, DEU_Battery_Discharge_amount, discharge_efficiency)]


In [ ]:
#### Then just pick the data you want and expand the length from 7 to 35 (as in this paper we need the line plot)

In [ ]:
### Plot Figure 8, S8 and S9

# New Plot

# New expanded datasets
benefits_data = [
                 Battery_UnitBenefit_full,
                 H207_UnitBenefit_full, 
                 H201_UnitBenefit_full, 
                 H202_UnitBenefit_full, 
                 H203_UnitBenefit_full]

invest_data = [
               Battery_UnitInvest_ac_full,
               H207_UnitInvest_full, 
               H201_UnitInvest_full, 
               H202_UnitInvest_full,
               H203_UnitInvest_full]

opt_data = [
            Battery_OPT_all_full_modified,
            H2_OPT_all_full,
            H2_OPT_ee_full, 
            H2_OPT_eh_full,
            H2_OPT_pe_full,
            H2_OPT_ph_full]

countries = ['DNK', 'DEU', 'ITA']

# Adjust the subplot grid to match the new data structure
fig, axs = plt.subplots(3, 4, figsize=(25, 15))  # Adjust the number of subplots if needed

# New technology titles

tech_titles = [
    'Battery', 'Indirect and Direct H$_{2}$ Storage',
    'ESFC', 'ESSE or ESSH',
    'ESE or ESH'
]

line_colors = [
    '#904840', '#9E7A7A',
    '#A35E47', '#08192D', '#78C2C4',
    '#734338', '#A96360',
    '#854836', '#26453D', '#305A56',
]


c_line_colors = [
    '#2B5F75', '#3F2B36',
    '#77428D', '#211E55',
    '#1B813E', '#A36336',
]

shade_colors = [
    '#AD8E70', '#FFEBB7',
    '#D8D8D8', '#408E91',
    '#FF6E31',
]


column_datasets = [
    (0,),
    (1,),
    (2,3,),
    (4,)
]

primary_y_max = 200
secondary_y_max = 0

for datasets in column_datasets:
    for dataset_index in datasets:
        max_benefit = max([item for sublist in benefits_data[dataset_index] for item in sublist])
        max_invest = max([item for sublist in invest_data[dataset_index] for item in sublist])

        if dataset_index < len(opt_data):
            max_opt = max([item for sublist in opt_data[dataset_index] for item in sublist])
            secondary_y_max = max(secondary_y_max, max_opt) + 1000
            
for i, country in enumerate(countries):
    for j in range(4):
        ax = axs[i, j]
        for dataset_index in column_datasets[j]:
            tech = tech_titles[dataset_index]

            if j == 0:
                
                ax.plot(range(2030, 2055), benefits_data[dataset_index][i][10:35], label=f'{tech} Unit Benefit', linewidth = 3, color=line_colors[dataset_index])
                ax.plot(range(2030, 2055), invest_data[dataset_index][i][10:35], label=f'{tech} LCOS', linewidth = 3, color=line_colors[dataset_index + 5])

                ax.fill_between(range(2030, 2055), benefits_data[dataset_index][i][10:35], invest_data[dataset_index][i][10:35], color=shade_colors[dataset_index], alpha=0.6)
                
                ax.set_xticks([2030, 2035, 2040, 2045, 2050, 2055])
                ax.set_xticklabels(['2030', '2035', '2040', '2045', '2050', '2055'])
                
                ax2 = ax.twinx()
                ax2.plot(range(2030, 2055), opt_data[0][i][10:35], label=f'Battery Capacity after \nConsider the Electricity Load', linestyle='--', linewidth = 3, color=c_line_colors[0])
                ax2.set_ylim(0, 1200000)
                ax2.set_ylabel("Battery Storage Capacity (MWh)")
                ax.set_title(f'{country} - Economic and Capacity Analysis of Battery')
                ax.legend(loc='upper left', bbox_to_anchor=(0.01, 0.89), frameon=False, fontsize=13)
                ax2.legend(loc='upper left', bbox_to_anchor=(0.01, 1), frameon=False, fontsize=13)
            
            if j == 1:
                
                ax.plot(range(2040, 2055), benefits_data[dataset_index][i][20:35], label=f'{tech} Unit Benefit', linewidth = 3, color=line_colors[dataset_index])
                ax.plot(range(2040, 2055), invest_data[dataset_index][i][20:35], label=f'{tech} LCOS', linewidth = 3, color=line_colors[dataset_index + 5])

                ax.fill_between(range(2040, 2055), benefits_data[dataset_index][i][20:35], invest_data[dataset_index][i][20:35], color=shade_colors[dataset_index], alpha=0.6)

                ax.set_xticks([2040, 2045, 2050, 2055])
                ax.set_xticklabels(['2040', '2045', '2050', '2055'])
                ax.set_title(f'{country} - Economic and Direct Capacity Analysis of all Hydorgen')
                
                ax2 = ax.twinx()
                ax2.plot(range(2040, 2055), opt_data[1][i][20:35], label=f'Direct H$_{2}$ Storage Capacity', linestyle='--', linewidth = 3, color=c_line_colors[1])
                ax2.set_ylim(0, secondary_y_max)
                ax2.set_ylabel("Direct Storage Capacity (MWh)")
                ax.legend(loc='upper left', bbox_to_anchor=(0.01, 0.94), frameon=False, fontsize=13)
                ax2.legend(loc='upper left', bbox_to_anchor=(0.01, 1), frameon=False, fontsize=13)
            
            if j == 2:
                
                ax.plot(range(2040, 2055), benefits_data[dataset_index][i][20:35], label=f'{tech} Unit Benefit', linewidth = 3, color=line_colors[dataset_index])
                ax.plot(range(2040, 2055), invest_data[dataset_index][i][20:35], label=f'{tech} LCOS', linewidth = 3, color=line_colors[dataset_index + 5])

                ax.fill_between(range(2040, 2055), benefits_data[dataset_index][i][20:35], invest_data[dataset_index][i][20:35], color=shade_colors[dataset_index], alpha=0.6)
                
                ax.set_xticks([2040, 2045, 2050, 2055])
                ax.set_xticklabels(['2040', '2045', '2050', '2055'])

                ax2 = ax.twinx()
                ax2.plot(range(2040, 2055), opt_data[2][i][20:35], label=f'ESSE Capacity', linestyle='--', linewidth = 3, color=c_line_colors[2])
                ax2.plot(range(2040, 2055), opt_data[3][i][20:35], label=f'ESSH Capacity', linestyle='--', linewidth = 3, color=c_line_colors[3])
                ax2.set_ylim(0, secondary_y_max)
                ax2.set_ylabel("Direct Storage Capacity (MWh)")
                ax.set_title(f'{country} - Economic and Capacity Analysis of Direct Hydorgen')
                ax.legend(loc='upper left', bbox_to_anchor=(0.01, 0.87), frameon=False, fontsize=13)
                ax2.legend(loc='upper left', bbox_to_anchor=(0.01, 1), frameon=False, fontsize=13)
                
            
            if j == 3:
            
                ax.plot(range(2040, 2055), benefits_data[dataset_index][i][20:35], label=f'{tech} Unit Benefit', linewidth = 3, color=line_colors[dataset_index])
                ax.plot(range(2040, 2055), invest_data[dataset_index][i][20:35], label=f'{tech} LCOS', linewidth = 3, color=line_colors[dataset_index + 5])
                
                ax.fill_between(range(2040, 2055), benefits_data[dataset_index][i][20:35], invest_data[dataset_index][i][20:35], color=shade_colors[dataset_index], alpha=0.6)
                
                ax.set_xticks([2040, 2045, 2050, 2055])
                ax.set_xticklabels(['2040', '2045', '2050', '2055'])
                
                ax2 = ax.twinx()
                ax2.plot(range(2040, 2055), opt_data[4][i][20:35], label=f'ESE Capacity', linestyle='--', linewidth = 3, color=c_line_colors[4])
                ax2.plot(range(2040, 2055), opt_data[5][i][20:35], label=f'ESH Capacity', linestyle='--', linewidth = 3, color=c_line_colors[5])
                ax2.set_ylim(0, 20000)
                ax2.set_ylabel("Indirect Storage Capacity (MW)")
                ax.set_title(f'{country} - Economic and Capacity Analysis of Indirect Hydrogen')
                ax.legend(loc='upper left', bbox_to_anchor=(0.01, 0.87), frameon=False, fontsize=13)
                ax2.legend(loc='upper left', bbox_to_anchor=(0.01, 1), frameon=False, fontsize=13)
                
                
        ax.set_ylim(0, primary_y_max)

        #ax.set_xlabel('Year')
        ax.set_ylabel('Unit Benefit/LCOS (€/MWh)')


plt.tight_layout()

#plt.savefig('Figure S08.png', dpi=300)

plt.show()


In [ ]:
#################### Calculate Storage Selling and Buying Price ##########################
##########################################################################################

In [ ]:
#################### Get the time index of storage

In [ ]:
# Prepare data

# DNK

# Direct for sabatier
DNK_Direct_2020_forsa_p1 = DNK_p_forsa[0]
DNK_Direct_2025_forsa_p1 = ...

# Indirect for sabatier
DNK_Indirect_2020_forsa_p1 = DNK_e_forsa[0]
DNK_Indirect_2025_forsa_p1 = ...

# Electrolysis
DNK_Electrolysis_2020_Output_p1 = -n_2020.links_t.p1["DK H2 Electrolysis"]
DNK_Electrolysis_2025_Output_p1 = ...

# Sabatier
DNK_Sabatier_2020_Input_p0 = n_2020.links_t.p0["DK Sabatier"]
DNK_Sabatier_2025_Input_p0 = ...

# DEU
...

# ITA
...


In [ ]:
# DNK, time index

years = [2020, 2025, 2030, 2035, 2040, 2045, 2050]
data_types = ['Electrolysis_Output_p1', 'Direct_forsa_p1', 'Indirect_forsa_p1', 'batterylink_charge_p0', 'batterylink_discharge_p1']


for year in years:
    for data_type in data_types:
        var_name = f"t{year}_DNK_{data_type}"
        original_series = globals()[f"DNK_{data_type.split('_')[0]}_{year}_{data_type.split('_')[1]}_{data_type.split('_')[2]}"]
        if isinstance(original_series, pd.Series):
            globals()[var_name] = original_series.apply(lambda x: 0 if abs(x) < 0.5 else 1)
        else:
            globals()[var_name] = original_series.applymap(lambda x: 0 if abs(x) < 0.5 else 1)

# DEU
...

#ITA
...


In [ ]:
#################### Get the original selling and buying price of storage

In [ ]:
# Get the original Price information, DNK

years = [2020, 2025, 2030, 2035, 2040, 2045, 2050]

price_data = {}

for year in years:
    price_charge_h2, price_direct_discharge_h2, price_indirect_discharge_h2, price_charge_battery, price_discharge_battery = create_price_variables(
        year,
        globals()[f't{year}_DNK_Electrolysis_Output_p1'],
        globals()[f't{year}_DNK_Direct_forsa_p1'],
        globals()[f't{year}_DNK_Indirect_forsa_p1'],
        globals()[f't{year}_DNK_batterylink_charge_p0'],
        globals()[f't{year}_DNK_batterylink_discharge_p1'],
        globals()[f'DNK_EleP_{year}'],
        globals()[f'DNK_H2_MarginalPrice_{year}']
    )
    globals()[f'Price{year}_charge_DNKH2P0'] = price_charge_h2
    globals()[f'Price{year}_direct_discharge_DNKH2P1'] = price_direct_discharge_h2
    globals()[f'Price{year}_indirect_discharge_DNKH2P1'] = price_indirect_discharge_h2
    globals()[f'Price{year}_charge_DNKBatteryP0'] = price_charge_battery
    globals()[f'Price{year}_discharge_DNKBatteryP1'] = price_discharge_battery

# DEU
...

#ITA
...


In [ ]:
# Plot Figure 6 and S6

fig, axs = plt.subplots(8, 3, figsize=(16.5, 18))

def plot_country_data(country_h2_data, country_battery_data, axs_col, country_label):
    
    for i in range(3):
        year = 2020 + 5 * (i+4)
        
        avg_charge = np.mean(country_h2_data[3*(i+4)])
        avg_direct_discharge = np.mean(country_h2_data[3*(i+4) + 1])
        avg_indirect_discharge = np.mean(country_h2_data[3*(i+4) + 2])
        
        labels = [
            f"Electrolysis Output Price (Avg: {avg_charge:.2f} €/MWh)", 
            f"ESE & ESH Mode Selling Price (Avg: {avg_direct_discharge:.2f} €/MWh)",
            f"ESSE & ESSH Mode Selling Price (Avg: {avg_indirect_discharge:.2f} €/MWh)"
        ]
        
        h2_data = country_h2_data[3*(i+4):3*(i+4)+3]
        axs[i, axs_col].hist(h2_data, bins=np.linspace(0, percentile_95, 61), stacked=True, color=[colors_H2['charge'], colors_H2['direct_discharge'], colors_H2['indirect_discharge']], label=labels)
        axs[i, axs_col].set_title(f"{country_label} H$_{2}$ {year}")
        axs[i, axs_col].set_xticklabels([])
        axs[i, axs_col].set_ylim(0,6000)
        
        axs[i, axs_col].legend(loc='upper left', frameon=False)
        
    for i in range(3, 8):
        year = 2030 + 5 * (i - 3)
        
        avg_charge = np.mean(country_battery_data[3*(i-1)])
        avg_direct_discharge = np.mean(country_battery_data[3*(i-1) + 1])
        avg_indirect_discharge = np.mean(country_battery_data[3*(i-1) + 2])
        
        labels = [
            f"Battery Charging Price (Avg: {avg_charge:.2f} €/MWh)", 
            f"Battery Discharging Price (Avg: {avg_discharge:.2f} €/MWh)",
            f"Electricity Price (Avg: {avg_ele_price:.2f} €/MWh)"
        ]
    
        battery_data = country_battery_data[3*(i-1):3*i]
        axs[i, axs_col].hist(battery_data, bins=np.linspace(0, percentile_95, 61), stacked=True, color=[colors_Battery['charge'], colors_Battery['discharge'], colors_Battery['ele_price']], label = labels)
        axs[i, axs_col].set_title(f"{country_label} Battery {year}")
        axs[i, axs_col].set_ylim(0,6000)
        if i < 7:
            axs[i, axs_col].set_xticklabels([])
        else:
            axs[i, axs_col].set_xlabel('Price (€/MWh)')
        # Set legends
        axs[i, axs_col].legend(loc='upper left', frameon=False)

plot_country_data(DNK_H2_pricedata_set, DNK_Battery_pricedata_set, 0, 'DNK')
plot_country_data(DEU_H2_pricedata_set, DEU_Battery_pricedata_set, 1, 'DEU')
plot_country_data(ITA_H2_pricedata_set, ITA_Battery_pricedata_set, 2, 'ITA')

plt.tight_layout()

plt.savefig('Figure S06.png', dpi=300)

plt.show()

In [ ]:
######################## Price Spread and Frequency Plot #################################
##########################################################################################

In [ ]:
# Prepare the data 

# Frequency, use the cycle number over a year as frequency

# DNK
DNK_H2_Filling_2020, DNK_Battery_Filling_2020 = filling_01(n_2020.stores_t.e["DK H2 Store tank"],n_2020.stores_t.e["DK H2 Store underground"],n_2020.stores_t.e["DK battery"],n_2020.stores.e_nom_opt["DK H2 Store tank"],n_2020.stores.e_nom_opt["DK H2 Store underground"],n_2020.stores.e_nom_opt["DK battery"])
DNK_H2_Filling_2025, DNK_Battery_Filling_2025 = ...

# DEU
...

# ITA
...

def normalize_series(series):
    min_val = series.min()
    max_val = series.max()
    normalized = (series - min_val) / (max_val - min_val)
    return normalized

normalized_DNK_H2_FuelCell_amount = [normalize_series(series) for series in DNK_H2_FuelCell_amount]
normalized_DNK_p_forsa = [normalize_series(series) for series in DNK_p_forsa]
normalized_DNK_e_forsa = [normalize_series(series) for series in DNK_e_forsa]

...


In [ ]:
# Capture the Cycle Information

threshold_01 = 0.1
threshold_02 = 0.1

datasets = [
    DNK_Batteryall_series, DEU_Batteryall_series, ITA_Batteryall_series, 
    normalized_DNK_H2_FuelCell_amount, normalized_DNK_p_forsa, normalized_DNK_e_forsa, 
    normalized_DEU_H2_FuelCell_amount, normalized_DEU_p_forsa, normalized_DEU_e_forsa, 
    normalized_ITA_H2_FuelCell_amount, normalized_ITA_p_forsa, normalized_ITA_e_forsa
]

dataset_names = [
    "DNK_Batteryall", "DEU_Batteryall", "ITA_Batteryall", 
    "normalized_DNK_H2_FuelCell", "normalized_DNK_p_forsa", "normalized_DNK_e_forsa", 
    "normalized_DEU_H2_FuelCell", "normalized_DEU_p_forsa", "normalized_DEU_e_forsa", 
    "normalized_ITA_H2_FuelCell", "normalized_ITA_p_forsa", "normalized_ITA_e_forsa"
]

results = {}

for dataset, dataset_name in zip(datasets, dataset_names):
    for year, series in enumerate(dataset, start=2020):
        if year in [2020, 2025, 2030, 2035, 2040, 2045, 2050]:
            cycles, new_series, state_series = calculate_all_cycles(series, threshold_01, threshold_02)
            
            if dataset_name not in results:
                results[dataset_name] = {}
            results[dataset_name][year] = (cycles, new_series, state_series)


In [ ]:
## prepare ratio to calculate the charging cost

# DNK
DNK_H2_ChargingCost_all = [(x * y["DK"]) for x, y in zip(DNK_H2_Production_amount_all, DNK_EleP)]

DNK_ratio_fuelcell = [(x/H2_Around) / y for x, y in zip(DNK_H2_FuelCell_amount, DNK_H2_Production_amount_all)]
DNK_ratio_eforsa = [(x/Electrolysis_efficiency) / y for x, y in zip(DNK_e_forsa, DNK_H2_Production_amount_all)]
DNK_ratio_pforsa = [(x/Electrolysis_efficiency) / y for x, y in zip(DNK_p_forsa, DNK_H2_Production_amount_all)]

DNK_ratio_fuelcell = [series.clip(lower=0, upper=1) for series in DNK_ratio_fuelcell]
DNK_ratio_eforsa = [series.clip(lower=0, upper=1) for series in DNK_ratio_eforsa]
DNK_ratio_pforsa = [series.clip(lower=0, upper=1) for series in DNK_ratio_pforsa]

DNK_fuelcell_cc = [x * y for x, y in zip(DNK_ratio_fuelcell, DNK_H2_ChargingCost_all)]
DNK_e_cc = [x * y for x, y in zip(DNK_ratio_eforsa, DNK_H2_ChargingCost_all)]
DNK_p_cc = [x * y for x, y in zip(DNK_ratio_pforsa, DNK_H2_ChargingCost_all)]

# DEU
...

# ITA
...


In [ ]:
# Combine the frequency and price spread information in a result dict

results_summary = {}

for dataset_name in dataset_names:
    for index, year in enumerate(processing_years):
        total_ele_charging, total_ele_discharging = 0, 0
        total_h2_charging, total_h2_discharging = 0, 0
        total_battery_charging_price, total_battery_discharging_price = 0, 0
        total_fuelcell_charging_price, total_fuelcell_discharging_price = 0, 0
        total_e_charging_price, total_e_discharging_price = 0, 0 
        total_p_charging_price, total_p_discharging_price = 0, 0 
            
        num_cycles = len(results[dataset_name][year][0])

        for cycle in results[dataset_name][year][0]:
            start, end = cycle['start'], cycle['end']

            ele_charging_prices, ele_discharging_prices = [], []
            h2_charging_prices, h2_discharging_prices = [], []
            
            battery_charging_a, battery_charging_c, battery_discharging_a, battery_discharging_c = [], [], [], []
            
            fuelcell_charging_a, fuelcell_charging_c, fuelcell_discharging_a, fuelcell_discharging_c = [], [], [], []
            e_charging_a, e_charging_c, e_discharging_a, e_discharging_c = [], [], [], []
            p_charging_a, p_charging_c, p_discharging_a, p_discharging_c = [], [], [], []
            
            for i in range(start, end + 1):
                state = results[dataset_name][year][2].iloc[i]
                
                if "DNK" in dataset_name:
                    
                    ele_price = DNK_EleP[index]["DK"].iloc[i]
                    h2_price = DNK_H2_MarginalPrice[index].iloc[i]
                
                    # battery charging / dischaging amonut / cost
                    battery_c_a = DNK_Battery_Charge_amount[index][i]
                    battery_d_a = DNK_Battery_Discharge_amount[index][i]

                    battery_c = battery_c_a * ele_price
                    battery_d = battery_d_a * ele_price

                    # h2 discharging amount and cost

                    fuelcell_d_a = DNK_H2_FuelCell_amount[index][i]
                    e_d_a = DNK_e_forsa[index][i]
                    p_d_a = DNK_p_forsa[index][i]

                    fuelcell_d = max(0, fuelcell_d_a * ele_price)
                    e_d = max(0, e_d_a * h2_price)
                    p_d = max(0, p_d_a * h2_price)

                    # h2 charging amount and cost
                    fuelcell_c_a = fuelcell_d_a / H2_Around
                    e_c_a = e_d_a / Electrolysis_efficiency
                    p_c_a = p_d_a / Electrolysis_efficiency

                    fuelcell_c = DNK_fuelcell_cc[index][i]
                    e_c = DNK_e_cc[index][i]
                    p_c = DNK_p_cc[index][i]
                    
                if "DEU" in dataset_name:
                    
                    ele_price = DEU_EleP[index]["DE"].iloc[i]
                    h2_price = DEU_H2_MarginalPrice[index].iloc[i]
                    
                    # battery charging / dischaging amonut / cost
                    battery_c_a = DEU_Battery_Charge_amount[index][i]
                    battery_d_a = DEU_Battery_Discharge_amount[index][i]

                    battery_c = battery_c_a * ele_price
                    battery_d = battery_d_a * ele_price

                    # h2 discharging amount and cost

                    fuelcell_d_a = DEU_H2_FuelCell_amount[index][i]
                    e_d_a = DEU_e_forsa[index][i]
                    p_d_a = DEU_p_forsa[index][i]

                    fuelcell_d = max(0, fuelcell_d_a * ele_price)
                    e_d = max(0, e_d_a * h2_price)
                    p_d = max(0, p_d_a * h2_price)

                    # h2 charging amount and cost
                    fuelcell_c_a = fuelcell_d_a / H2_Around
                    e_c_a = e_d_a / Electrolysis_efficiency
                    p_c_a = p_d_a / Electrolysis_efficiency

                    fuelcell_c = DEU_fuelcell_cc[index][i]
                    e_c = DEU_e_cc[index][i]
                    p_c = DEU_p_cc[index][i]
                    
                if "ITA" in dataset_name:
                    
                    ele_price = ITA_EleP[index]["IT"].iloc[i]
                    h2_price = ITA_H2_MarginalPrice[index].iloc[i]
                    
                    battery_c_a = ITA_Battery_Charge_amount[index][i]
                    battery_d_a = ITA_Battery_Discharge_amount[index][i]

                    battery_c = battery_c_a * ele_price
                    battery_d = battery_d_a * ele_price

                    fuelcell_d_a = ITA_H2_FuelCell_amount[index][i]
                    e_d_a = ITA_e_forsa[index][i]
                    p_d_a = ITA_p_forsa[index][i]

                    fuelcell_d = max(0, fuelcell_d_a * ele_price)
                    e_d = max(0, e_d_a * h2_price)
                    p_d = max(0, p_d_a * h2_price)

                    fuelcell_c_a = fuelcell_d_a / H2_Around
                    e_c_a = e_d_a / Electrolysis_efficiency
                    p_c_a = p_d_a / Electrolysis_efficiency

                    fuelcell_c = ITA_fuelcell_cc[index][i]
                    e_c = ITA_e_cc[index][i]
                    p_c = ITA_p_cc[index][i]

                if state == 'charging':
                    ele_charging_prices.append(ele_price)
                    h2_charging_prices.append(h2_price)
                    
                    battery_charging_a.append(battery_c_a)
                    battery_charging_c.append(battery_c)
                    
                    fuelcell_charging_a.append(fuelcell_c_a)
                    e_charging_a.append(e_c_a)
                    p_charging_a.append(p_c_a)
                    
                    fuelcell_charging_c.append(fuelcell_c)
                    e_charging_c.append(e_c)
                    p_charging_c.append(p_c)
                    
                elif state == 'discharging':
                    ele_discharging_prices.append(ele_price)
                    h2_discharging_prices.append(h2_price)
                    
                    battery_discharging_a.append(battery_d_a)
                    battery_discharging_c.append(battery_d)
                    
                    fuelcell_discharging_a.append(fuelcell_d_a)
                    e_discharging_a.append(e_d_a)
                    p_discharging_a.append(p_d_a)
                    
                    fuelcell_discharging_c.append(fuelcell_d)
                    e_discharging_c.append(e_d)
                    p_discharging_c.append(p_d)

            avg_ele_charging_price = sum(ele_charging_prices) / len(ele_charging_prices) if ele_charging_prices else 0
            avg_ele_discharging_price = sum(ele_discharging_prices) / len(ele_discharging_prices) if ele_discharging_prices else 0
            avg_h2_charging_price = sum(h2_charging_prices) / len(h2_charging_prices) if h2_charging_prices else 0
            avg_h2_discharging_price = sum(h2_discharging_prices) / len(h2_discharging_prices) if h2_discharging_prices else 0

            total_ele_charging += avg_ele_charging_price
            total_ele_discharging += avg_ele_discharging_price
            total_h2_charging += avg_h2_charging_price
            total_h2_discharging += avg_h2_discharging_price
            
            avg_battery_charging_price = sum(battery_charging_c) / sum(battery_charging_a)
            avg_battery_discharging_price = sum(battery_discharging_c) / sum(battery_discharging_a)
            avg_fuelcell_charging_price = sum(fuelcell_charging_c) / sum(fuelcell_charging_a)
            avg_fuelcell_discharging_price = sum(fuelcell_discharging_c) / sum(fuelcell_discharging_a)
            avg_e_charging_price = sum(e_charging_c) / sum(e_charging_a)
            avg_e_discharging_price = sum(e_discharging_c) / sum(e_discharging_a)
            avg_p_charging_price = sum(p_charging_c) / sum(p_charging_a)
            avg_p_discharging_price = sum(p_discharging_c) / sum(p_discharging_a)
            
            total_battery_charging_price += avg_battery_charging_price
            total_battery_discharging_price += avg_battery_discharging_price
            total_fuelcell_charging_price += avg_fuelcell_charging_price
            total_fuelcell_discharging_price += avg_fuelcell_discharging_price
            total_e_charging_price += avg_e_charging_price
            total_e_discharging_price += avg_e_discharging_price
            total_p_charging_price += avg_p_charging_price
            total_p_discharging_price += avg_p_discharging_price

        overall_avg_ele_charging = total_ele_charging / num_cycles
        overall_avg_ele_discharging = total_ele_discharging / num_cycles
        overall_avg_h2_charging = total_h2_charging / num_cycles
        overall_avg_h2_discharging = total_h2_discharging / num_cycles
        
        avg_ele_price_spread = overall_avg_ele_discharging - overall_avg_ele_charging
        avg_h2_price_spread = overall_avg_h2_discharging - overall_avg_h2_charging
        
        avg_battery_price_spread = max(0, (total_battery_discharging_price - total_battery_charging_price) / num_cycles)
        avg_fuelcell_price_spread = max(0, (total_fuelcell_discharging_price - total_fuelcell_charging_price) / num_cycles)
        avg_e_price_spread = max(0, (total_e_discharging_price - total_e_charging_price) / num_cycles)
        avg_p_price_spread = max(0, (total_p_discharging_price - total_p_charging_price) / num_cycles)
        
        if "Battery" in dataset_name:
            results_summary[f"{dataset_name}_{year}_price_spread"] = avg_battery_price_spread
            results_summary[f"{dataset_name}_{year}_num_cycles"] = num_cycles
        elif "FuelCell" in dataset_name:
            results_summary[f"{dataset_name}_{year}_price_spread"] = avg_fuelcell_price_spread
            results_summary[f"{dataset_name}_{year}_num_cycles"] = num_cycles
        elif "e_forsa" in dataset_name:
            results_summary[f"{dataset_name}_{year}_price_spread"] = avg_e_price_spread
            results_summary[f"{dataset_name}_{year}_num_cycles"] = num_cycles
        elif "p_forsa" in dataset_name:
            results_summary[f"{dataset_name}_{year}_price_spread"] = avg_p_price_spread
            results_summary[f"{dataset_name}_{year}_num_cycles"] = num_cycles


In [ ]:
## Then just distribute the resluts with the variable name you desired

In [ ]:
# Plot Figure 7 and S7

years = [2020, 2025, 2030, 2035, 2040, 2045, 2050]
rainbow_colors = ['#9400D3', '#4B0082', '#0000FF', '#00FF00', '#FFFF00', '#FF7F00', '#FF0000']

countries = ['DNK', 'DEU', 'ITA']

frequencies = [
    [DNK_e_cycle_frequency, DNK_p_cycle_frequency, DNK_Battery_cycle_frequency],
    [DEU_e_cycle_frequency, DEU_p_cycle_frequency, DEU_Battery_cycle_frequency],
    [ITA_e_cycle_frequency, ITA_p_cycle_frequency, ITA_Battery_cycle_frequency]
]

spreads = [
    [DNK_e_cycle_ps, DNK_p_cycle_ps, DNK_Battery_cycle_ps],
    [DEU_e_cycle_ps, DEU_p_cycle_ps, DEU_Battery_cycle_ps],
    [ITA_e_cycle_ps, ITA_p_cycle_ps, ITA_Battery_cycle_ps]
]

dir_color, ind_color = '#F1EB90', '#F3B664'
markers = ['o', 'x', 's', '^']

global_min = min([min([min(data) for data in sublist]) for sublist in spreads])
global_max = max([max([max(data) for data in sublist]) for sublist in spreads])

fig, axs = plt.subplots(3, 1, figsize=(10, 15))

scatter_legend_entries = [mlines.Line2D([], [], color=year_color, marker='o', linestyle='None', label=str(year)) for year_color, year in zip(rainbow_colors, years)]

shade_legend_entries = [
    mpatches.Patch(color='#F1EB90', label='Part of Direct H$_{2}$ Storage (ESSE + ESSH)'),
    mpatches.Patch(color='#EC8F5E', label='Indirect H$_{2}$ storage (ESE + ESH)'),
    mpatches.Patch(color='#9FBB73', label='Battery')
]

total_entries = len(scatter_legend_entries) + len(shade_legend_entries)

for i, ax in enumerate(axs):
    for j, year_color in enumerate(rainbow_colors):
        
        ax.scatter(frequencies[i][0][j], spreads[i][0][j], color=year_color, marker=markers[2])
        ax.scatter(frequencies[i][1][j], spreads[i][1][j], color=year_color, marker=markers[3])
        ax.scatter(frequencies[i][2][j], spreads[i][2][j], color=year_color, marker=markers[1]) 

    ax.set_title(f"{countries[i]} Frequency and Price Spread plot")
    
    ax.set_ylabel("Price Spread (€/MWh)")
    ax.set_ylim(global_min, global_max)
    ax.set_xlim(0, 700)
    
    if i >1:
        ax.set_xlabel("Frequency (The number of storage filling level cycle)")

    for k in range(2):
        points = np.column_stack((frequencies[i][k], spreads[i][k]))
        hull = ConvexHull(points)
        shade_color = dir_color if k == 0 else (ind_color if k == 1 else '#EC8F5E')
        polygon = Polygon(points[hull.vertices], closed=True, color=shade_color, alpha=0.4)
        ax.add_patch(polygon)

    points_Battery = np.column_stack((frequencies[i][2], spreads[i][2]))
    hull_Battery = ConvexHull(points_Battery)
    polygon_Battery = Polygon(points_Battery[hull_Battery.vertices], closed=True, color='#9FBB73', alpha=0.6)
    ax.add_patch(polygon_Battery)

scatter_legend = fig.legend(handles=scatter_legend_entries, loc='lower center', bbox_to_anchor=(0.5, 0.12), ncol=len(scatter_legend_entries), frameon=False)

shade_legend = fig.legend(handles=shade_legend_entries, loc='lower center', bbox_to_anchor=(0.5, 0.14), ncol=len(shade_legend_entries), frameon=False)

plt.tight_layout()

plt.subplots_adjust(bottom = 0.2)

plt.savefig('Figure S07.png', dpi=300)

plt.show()

In [ ]:
############################ H2 Each Part Revenue Plot ###################################
##########################################################################################

In [ ]:
# Plot Figure 09 and S10

# Your data
DNK_data = [DNK_H2_DC_fc[4:7], DNK_H2_DC_ee[4:7], DNK_H2_DC_eh[4:7], DNK_H2_DC_pe[4:7], DNK_H2_DC_ph[4:7]]
DEU_data = [DEU_H2_DC_fc[4:7], DEU_H2_DC_ee[4:7], DEU_H2_DC_eh[4:7], DEU_H2_DC_pe[4:7], DEU_H2_DC_ph[4:7]]
ITA_data = [ITA_H2_DC_fc[4:7], ITA_H2_DC_ee[4:7], ITA_H2_DC_eh[4:7], ITA_H2_DC_pe[4:7], ITA_H2_DC_ph[4:7]]
all_data = [DNK_data, DEU_data, ITA_data]

# Years and country names
years = [2040, 2045, 2050]
countries = ['Denmark', 'Germany', 'Italy']

custom_colors = ['#1AACAC', '#001253', '#FD841F', '#3E6D9C', '#E14D2A']

# Labels for the pie chart segments
legend_labels = ['ESFC (Hydrogen from Storage to Fuel Cell)', 
                 'ESSE (Hydrogen from Storage to Sabatier and finally to Generate Electricity)', 
                 'ESSH (Hydrogen from Storage to Sabatier and finally to Generate Heating)', 
                 'ESE (Hydrogen from Electrolysis to Sabatier and finally to Generate Electricity)', 
                 'ESH (Hydrogen from Electrolysis to Sabatier and finally to Generate Heating)']

labels = ['ESFC', 
          'ESSE', 
          'ESSH', 
          'ESE', 
          'ESH']

def custom_autopct(pct):
    return ('%.1f%%' % pct) if pct >= 3 else ''

fig, axs = plt.subplots(3, 3, figsize=(11, 10))

for i in range(3):
    for j in range(3):

        year_data = [country_data[j] for country_data in all_data[i]]
        total_value = sum(year_data)
        
        percentages = [x / total_value * 100 for x in year_data]

        custom_labels = [labels[k] if percentages[k] >= 3 else '' for k in range(len(labels))]

        axs[i, j].pie(year_data, autopct=custom_autopct, startangle=140, labels=custom_labels, colors=custom_colors)
        axs[i, j].set_title(f'{countries[i]} {years[j]} \nTotal: {total_value / 1e6:.2f} x 10^6 €', wrap=True)

plt.figlegend(legend_labels, loc='upper center', bbox_to_anchor=(0.5, 1), ncol=1, frameon=False)

fig.subplots_adjust(hspace=0.2, wspace = 0.00000001, top=0.8)

plt.savefig('Figure S10.png', dpi=300)

plt.show()


In [ ]:
############################ FFT, CWT, CC Analysis ###################################
############################ Including Electricity Price, Direct & Indirect H2 & Battery change

In [ ]:
########### Take Electricity Price as Example, others are same

In [ ]:
##### Plot FFT of Electricity Price

def create_ele_combine_plot(data_list, data_labels, filename, country_codes):
    fig, axs = plt.subplots(7, len(data_list), figsize=(15,20))

    for i, (data, country_code) in enumerate(zip(data_list, country_codes)):
        for j, series in enumerate(data):
            data=series.copy()
            data.reset_index(drop = True,inplace=True)

            dt=1
            freqs1, ps1, psd1 = spectrum1(data[country_code]/data[country_code].mean(), dt=dt)

            axs[j,i].set_ylim([0,1])
            axs[j,i].semilogx(freqs1, ps1/ps1.max(), 'green')
            
            if i == 0:
                axs[j,i].set_title('Power Spectrum for DNK {} EleP'.format(data_labels[j]))
            elif i == 1:
                axs[j,i].set_title('Power Spectrum for DEU {} EleP'.format(data_labels[j]))
            elif i == 2:
                axs[j,i].set_title('Power Spectrum for ITA {} EleP'.format(data_labels[j]))

            axs[j,i].axvline(x = 24, c="lightgrey", ls="--", lw=1) 
            axs[j,i].axvline(x = 168, c="lightgrey", ls="--", lw=1) 
            axs[j,i].axvline(x = 720, c="lightgrey", ls="--", lw=1) 
            axs[j,i].axvline(x = 2160, c="lightgrey", ls="--", lw=1)

    for ax in axs.flat:
        ax.set_xlabel('(hours)')
        ax.set_ylabel('Capacity factor')

    fig.subplots_adjust(hspace=0.5, wspace=0.3)

    plt.savefig(filename, dpi=300)

# Define your data lists
DNK_data = [DNK_EleP_2020,DNK_EleP_2025,DNK_EleP_2030,DNK_EleP_2035,DNK_EleP_2040,DNK_EleP_2045,DNK_EleP_2050]
DEU_data = [DEU_EleP_2020,DEU_EleP_2025,DEU_EleP_2030,DEU_EleP_2035,DEU_EleP_2040,DEU_EleP_2045,DEU_EleP_2050]
ITA_data = [ITA_EleP_2020,ITA_EleP_2025,ITA_EleP_2030,ITA_EleP_2035,ITA_EleP_2040,ITA_EleP_2045,ITA_EleP_2050]

# Define your data labels
data_labels = ['2020', '2025', '2030', '2035', '2040', '2045', '2050']

# Call your function with the appropriate arguments

create_ele_combine_plot([DNK_data, DEU_data, ITA_data], data_labels, 'Combined_EleP_FFT.png', ['DK', 'DE', 'IT'])


In [ ]:
# Plot the CWT of Electricity Price

dt = 1

f0 = 1

scales_min = 1
scales_max = 8760
scales = np.logspace(np.log2(scales_min), np.log2(scales_max), num=100, base=2)

fig, axes = plt.subplots(nrows=3, ncols=7, figsize=(21, 9), sharex=True, sharey=True)

for i, ax in enumerate(axes.flat):
    data = CWT_data_list[i]
    
    cwt_result = cwt(data, morlet2, scales, w=f0)
    log_cwt_result = np.log10(np.abs(cwt_result))

    img = ax.imshow(log_cwt_result, aspect='auto', cmap='jet', extent=[0, len(data) * dt, np.log2(np.max(scales)), np.log2(np.min(scales))])
    ax.set_xlabel('Time (hours)')
    ax.set_title(CWT_data_titles[i])
    ax.set_yticks(np.log2(scales[::10]))
    ax.set_yticklabels(np.round(scales[::10], 1))
    
    if i % 7 == 0:
        ax.set_ylabel('Log Scale')
    
    ax.tick_params(axis='y', labelsize=9)
    
fig.subplots_adjust(right=0.9,hspace=0.4)
cbar_ax = fig.add_axes([0.92, 0.15, 0.02, 0.7])
fig.colorbar(img, cax=cbar_ax, label='Log Magnitude')
plt.suptitle("The frequency change in the year scale (Log)", fontsize=20, y=0.95)

plt.savefig('PyPSA Log EP Year time CWT analysis.png', dpi=300)

plt.show()

In [ ]:
# Plot the CC of Electricity Price

def extract_cycle_lengths(series_list, threshold_01, threshold_02):
    all_cycle_lengths = []
    
    for series in series_list:
        cycles, _ , _ = calculate_all_cycles(series, threshold_01, threshold_02)
        cycle_lengths = [cycle['length'] for cycle in cycles]
        all_cycle_lengths.append(cycle_lengths)
    
    return all_cycle_lengths

DNK_cycle_lengths = extract_cycle_lengths(DNK_NEPall_series, 0.05, 0.05)
DEU_cycle_lengths = ...

max_cycle_length_DNK = np.percentile([item for sublist in DNK_cycle_lengths for item in sublist], 99)
max_cycle_length_DEU = ...

min_cycle_length_DNK = np.percentile([item for sublist in DNK_cycle_lengths for item in sublist], 1)
min_cycle_length_DEU = ...

num_bins = 50

fig, axs = plt.subplots(7, 3, figsize=(15, 20))

years = [2020, 2025, 2030, 2035, 2040, 2045, 2050]

max_y_val_DNK = 0
max_y_val_DEU = 0
max_y_val_ITA = 0

for i, year in enumerate(years):
    DNK_hist = axs[i, 0].hist(DNK_cycle_lengths[i], bins=np.linspace(min_cycle_length_DNK, max_cycle_length_DNK, num_bins))
    axs[i, 0].set_title(f'CC for DNK {year} EleP')
    axs[i, 0].set_xlabel('Cycle Length')
    axs[i, 0].set_ylabel('Frequency')
    axs[i, 0].legend([f'Number of cycles: {len(DNK_cycle_lengths[i])}'], frameon=False)
    max_y_val_DNK = max(max_y_val_DNK, max(DNK_hist[0]))
    
    DEU_hist = axs[i, 1].hist(DEU_cycle_lengths[i], bins=np.linspace(min_cycle_length_DEU, max_cycle_length_DEU, num_bins))
    axs[i, 1].set_title(f'CC for DEU {year} EleP')
    axs[i, 1].set_xlabel('Cycle Length')
    axs[i, 1].legend([f'Number of cycles: {len(DEU_cycle_lengths[i])}'], frameon=False)
    max_y_val_DEU = max(max_y_val_DEU, max(DEU_hist[0]))
    
    ITA_hist = axs[i, 2].hist(ITA_cycle_lengths[i], bins=np.linspace(min_cycle_length_ITA, max_cycle_length_ITA, num_bins))
    axs[i, 2].set_title(f'CC for ITA {year} EleP')
    axs[i, 2].set_xlabel('Cycle Length')
    axs[i, 2].legend([f'Number of cycles: {len(ITA_cycle_lengths[i])}'], frameon=False)
    max_y_val_ITA = max(max_y_val_ITA, max(ITA_hist[0]))

for i in range(7):
    axs[i, 0].set_ylim(0, max_y_val_DNK)
    axs[i, 1].set_ylim(0, max_y_val_DEU)
    axs[i, 2].set_ylim(0, max_y_val_ITA)

plt.tight_layout()

plt.savefig('Combined_EleP_CC.png', dpi=300)

plt.show()
